In [28]:
#Importacion de librerias
import pandas as pd
import numpy as np
import datetime as DT
import category_encoders as ce
from sklearn.preprocessing import LabelEncoder
import warnings

warnings.filterwarnings('ignore')


In [29]:
#Lectura de archivos CSV
train = pd.read_csv('../../DATA_TP2/train.csv')

In [39]:
#Nuevo dataframe
trainProcesado = pd.DataFrame()

#Copiamos la columna con el id
trainProcesado['id'] = train['id']
trainProcesado['idzona'] = train['idzona']



#Manejo de NaN, columnas numericas
train['garages'].fillna(0, inplace=True)
train['banos'].fillna(1, inplace=True)

promedio_antiguedad_x_ciudad = train.groupby('ciudad')['antiguedad'].transform('mean')
train['antiguedad'].fillna(promedio_antiguedad_x_ciudad, inplace=True)
train['antiguedad'].fillna(train['antiguedad'].mean(), inplace=True)

promedio_habitaciones_x_ciudad = train.groupby('ciudad')['habitaciones'].transform('mean')
train['habitaciones'].fillna(promedio_habitaciones_x_ciudad, inplace=True)
train['habitaciones'].fillna(train['habitaciones'].mean(), inplace=True)

promedio_metroscubierto_x_ciudad = train.groupby('ciudad')['metroscubiertos'].transform('mean')
train['metroscubiertos'].fillna(promedio_metroscubierto_x_ciudad, inplace=True)
train['metroscubiertos'].fillna(train['metroscubiertos'].mean(), inplace=True)

promedio_metrostotales_x_ciudad = train.groupby('ciudad')['metrostotales'].transform('mean')
train['metrostotales'].fillna(promedio_metrostotales_x_ciudad, inplace=True)
train['metrostotales'].fillna(train['metrostotales'].mean(), inplace=True)

#Manejo de NaN, columnas catedoricas
train['ciudad'] = train['ciudad'].fillna('Desconocida')
train['provincia'] = train['provincia'].fillna('Desconocida')
train['tipodepropiedad'] = train['tipodepropiedad'].fillna('Desconocido')

train['ciudad'] = train['ciudad'].astype('category')
train['provincia'] = train['provincia'].astype('category')
train['tipodepropiedad'] = train['tipodepropiedad'].astype('category')

#Asignacion de tipos de datos
trainProcesado['antiguedad'] = train['antiguedad'].astype('int64')
trainProcesado['habitaciones'] = train['habitaciones'].astype('int64')
trainProcesado['garages'] = train['garages'].astype('int64')
trainProcesado['banos'] = train['banos'].astype('int64')
trainProcesado['metroscubiertos'] = train['metroscubiertos'].astype('int64')
trainProcesado['metrostotales'] = train['metrostotales'].astype('int64')

trainProcesado['gimnasio'] = train['gimnasio'].astype('int64')
trainProcesado['usosmultiples'] = train['usosmultiples'].astype('int64')
trainProcesado['piscina'] = train['piscina'].astype('int64')
trainProcesado['escuelascercanas'] = train['escuelascercanas'].astype('int64')
trainProcesado['escuelascercanas'] = train['escuelascercanas'].astype('int64')
trainProcesado['centroscomercialescercanos'] = train['centroscomercialescercanos'].astype('int64')


#Solo nos quedamos con el anio de la fecha
trainProcesado['anio'] = pd.to_datetime(train['fecha'], infer_datetime_format=True).dt.year
trainProcesado['2012'] = 0
trainProcesado['2013'] = 0
trainProcesado['2014'] = 0
trainProcesado['2015'] = 0
trainProcesado['2016'] = 0
trainProcesado.loc[train['fecha'].str.contains('2012', na=False), '2012'] = 1
trainProcesado.loc[train['fecha'].str.contains('2013', na=False), '2013'] = 1
trainProcesado.loc[train['fecha'].str.contains('2014', na=False), '2014'] = 1
trainProcesado.loc[train['fecha'].str.contains('2015', na=False), '2015'] = 1
trainProcesado.loc[train['fecha'].str.contains('2016', na=False), '2016'] = 1

#Precio
trainProcesado['precio'] = train['precio']

In [40]:
#Para las columas categorias
#Las guardamos con Label Encoding
cat_features = ['ciudad', 'provincia', 'tipodepropiedad']
encoder = LabelEncoder()
encoded = train[cat_features].apply(encoder.fit_transform)

trainProcesado = trainProcesado.join(encoded)
encoded

ciudad  provincia  tipodepropiedad
0           93          9                0
1          371          9                3
2          768         15                2
3          862         11                2
4          852         15                0
...        ...        ...              ...
239995     862         11                2
239996     767         11                2
239997      93          9                0
239998     334          9                2
239999     206          9                0

[240000 rows x 3 columns]

In [41]:
#AGRUPAMIENTO POR ANTIGUEDAD (BANDAS)
trainProcesado['antiguedad_agrupada'] = 0

trainProcesado.loc[trainProcesado.antiguedad <= 5, 'antiguedad_agrupada'] = 0
trainProcesado.loc[trainProcesado.antiguedad > 95, 'antiguedad_agrupada'] = 100
trainProcesado.loc[((trainProcesado.antiguedad > 5) & (trainProcesado.antiguedad <= 15)), 'antiguedad_agrupada'] = 10
trainProcesado.loc[((trainProcesado.antiguedad > 15) & (trainProcesado.antiguedad <= 25)), 'antiguedad_agrupada'] = 20
trainProcesado.loc[((trainProcesado.antiguedad > 25) & (trainProcesado.antiguedad <= 35)), 'antiguedad_agrupada'] = 30
trainProcesado.loc[((trainProcesado.antiguedad > 35) & (trainProcesado.antiguedad <= 45)), 'antiguedad_agrupada'] = 40
trainProcesado.loc[((trainProcesado.antiguedad > 45) & (trainProcesado.antiguedad <= 55)), 'antiguedad_agrupada'] = 50
trainProcesado.loc[((trainProcesado.antiguedad > 55) & (trainProcesado.antiguedad <= 65)), 'antiguedad_agrupada'] = 60
trainProcesado.loc[((trainProcesado.antiguedad > 65) & (trainProcesado.antiguedad <= 75)), 'antiguedad_agrupada'] = 70
trainProcesado.loc[((trainProcesado.antiguedad > 75) & (trainProcesado.antiguedad <= 85)), 'antiguedad_agrupada'] = 80
trainProcesado.loc[((trainProcesado.antiguedad > 85) & (trainProcesado.antiguedad <= 95)), 'antiguedad_agrupada'] = 90
# AGRUPAMIENTO POR METROS CUBIENTOS (BANDAS)
trainProcesado['metroscubiertos_agrupada'] = 0

trainProcesado.drop('antiguedad', 1, inplace = True)

trainProcesado.loc[trainProcesado.metroscubiertos <= 5, 'metroscubiertos_agrupada'] = 0
trainProcesado.loc[trainProcesado.metroscubiertos > 805, 'metroscubiertos_agrupada'] = 1000
trainProcesado.loc[((trainProcesado.metroscubiertos > 5) & (trainProcesado.metroscubiertos <= 15)), 'metroscubiertos_agrupada'] = 10
trainProcesado.loc[((trainProcesado.metroscubiertos > 15) & (trainProcesado.metroscubiertos <= 25)), 'metroscubiertos_agrupada'] = 20
trainProcesado.loc[((trainProcesado.metroscubiertos > 25) & (trainProcesado.metroscubiertos <= 35)), 'metroscubiertos_agrupada'] = 30
trainProcesado.loc[((trainProcesado.metroscubiertos > 35) & (trainProcesado.metroscubiertos <= 45)), 'metroscubiertos_agrupada'] = 40
trainProcesado.loc[((trainProcesado.metroscubiertos > 45) & (trainProcesado.metroscubiertos <= 55)), 'metroscubiertos_agrupada'] = 50
trainProcesado.loc[((trainProcesado.metroscubiertos > 55) & (trainProcesado.metroscubiertos <= 65)), 'metroscubiertos_agrupada'] = 60
trainProcesado.loc[((trainProcesado.metroscubiertos > 65) & (trainProcesado.metroscubiertos <= 75)), 'metroscubiertos_agrupada'] = 70
trainProcesado.loc[((trainProcesado.metroscubiertos > 75) & (trainProcesado.metroscubiertos <= 85)), 'metroscubiertos_agrupada'] = 80
trainProcesado.loc[((trainProcesado.metroscubiertos > 85) & (trainProcesado.metroscubiertos <= 95)), 'metroscubiertos_agrupada'] = 90
trainProcesado.loc[((trainProcesado.metroscubiertos > 95) & (trainProcesado.metroscubiertos <= 105)), 'metroscubiertos_agrupada'] = 100
trainProcesado.loc[((trainProcesado.metroscubiertos > 105) & (trainProcesado.metroscubiertos <= 115)), 'metroscubiertos_agrupada'] = 110
trainProcesado.loc[((trainProcesado.metroscubiertos > 115) & (trainProcesado.metroscubiertos <= 130)), 'metroscubiertos_agrupada'] = 125
trainProcesado.loc[((trainProcesado.metroscubiertos > 130) & (trainProcesado.metroscubiertos <= 155)), 'metroscubiertos_agrupada'] = 150
trainProcesado.loc[((trainProcesado.metroscubiertos > 155) & (trainProcesado.metroscubiertos <= 205)), 'metroscubiertos_agrupada'] = 200
trainProcesado.loc[((trainProcesado.metroscubiertos > 205) & (trainProcesado.metroscubiertos <= 255)), 'metroscubiertos_agrupada'] = 250
trainProcesado.loc[((trainProcesado.metroscubiertos > 255) & (trainProcesado.metroscubiertos <= 305)), 'metroscubiertos_agrupada'] = 300
trainProcesado.loc[((trainProcesado.metroscubiertos > 305) & (trainProcesado.metroscubiertos <= 355)), 'metroscubiertos_agrupada'] = 350
trainProcesado.loc[((trainProcesado.metroscubiertos > 355) & (trainProcesado.metroscubiertos <= 405)), 'metroscubiertos_agrupada'] = 400
trainProcesado.loc[((trainProcesado.metroscubiertos > 405) & (trainProcesado.metroscubiertos <= 455)), 'metroscubiertos_agrupada'] = 450
trainProcesado.loc[((trainProcesado.metroscubiertos > 455) & (trainProcesado.metroscubiertos <= 500)), 'metroscubiertos_agrupada'] = 500
trainProcesado.loc[((trainProcesado.metroscubiertos > 505) & (trainProcesado.metroscubiertos <= 555)), 'metroscubiertos_agrupada'] = 550
trainProcesado.loc[((trainProcesado.metroscubiertos > 555) & (trainProcesado.metroscubiertos <= 600)), 'metroscubiertos_agrupada'] = 600
trainProcesado.loc[((trainProcesado.metroscubiertos > 605) & (trainProcesado.metroscubiertos <= 655)), 'metroscubiertos_agrupada'] = 650
trainProcesado.loc[((trainProcesado.metroscubiertos > 655) & (trainProcesado.metroscubiertos <= 700)), 'metroscubiertos_agrupada'] = 700
trainProcesado.loc[((trainProcesado.metroscubiertos > 705) & (trainProcesado.metroscubiertos <= 755)), 'metroscubiertos_agrupada'] = 750
trainProcesado.loc[((trainProcesado.metroscubiertos > 755) & (trainProcesado.metroscubiertos <= 805)), 'metroscubiertos_agrupada'] = 800

trainProcesado.drop('metroscubiertos', 1, inplace = True)

# AGRUPAMIENTO POR METROS TOTALES (BANDAS)
trainProcesado['metrostotales_agrupada'] = 0

trainProcesado.loc[trainProcesado.metrostotales <= 5, 'metrostotales_agrupada'] = 0
trainProcesado.loc[trainProcesado.metrostotales > 805, 'metrostotales_agrupada'] = 1000
trainProcesado.loc[((trainProcesado.metrostotales > 5) & (trainProcesado.metrostotales <= 15)), 'metrostotales_agrupada'] = 10
trainProcesado.loc[((trainProcesado.metrostotales > 15) & (trainProcesado.metrostotales <= 25)), 'metrostotales_agrupada'] = 20
trainProcesado.loc[((trainProcesado.metrostotales > 25) & (trainProcesado.metrostotales <= 35)), 'metrostotales_agrupada'] = 30
trainProcesado.loc[((trainProcesado.metrostotales > 35) & (trainProcesado.metrostotales <= 45)), 'metrostotales_agrupada'] = 40
trainProcesado.loc[((trainProcesado.metrostotales > 45) & (trainProcesado.metrostotales <= 55)), 'metrostotales_agrupada'] = 50
trainProcesado.loc[((trainProcesado.metrostotales > 55) & (trainProcesado.metrostotales <= 65)), 'metrostotales_agrupada'] = 60
trainProcesado.loc[((trainProcesado.metrostotales > 65) & (trainProcesado.metrostotales <= 75)), 'metrostotales_agrupada'] = 70
trainProcesado.loc[((trainProcesado.metrostotales > 75) & (trainProcesado.metrostotales <= 85)), 'metrostotales_agrupada'] = 80
trainProcesado.loc[((trainProcesado.metrostotales > 85) & (trainProcesado.metrostotales <= 95)), 'metrostotales_agrupada'] = 90
trainProcesado.loc[((trainProcesado.metrostotales > 95) & (trainProcesado.metrostotales <= 105)), 'metrostotales_agrupada'] = 100
trainProcesado.loc[((trainProcesado.metrostotales > 105) & (trainProcesado.metrostotales <= 115)), 'metrostotales_agrupada'] = 110
trainProcesado.loc[((trainProcesado.metrostotales > 115) & (trainProcesado.metrostotales <= 130)), 'metrostotales_agrupada'] = 125
trainProcesado.loc[((trainProcesado.metrostotales > 130) & (trainProcesado.metrostotales <= 155)), 'metrostotales_agrupada'] = 150
trainProcesado.loc[((trainProcesado.metrostotales > 155) & (trainProcesado.metrostotales <= 205)), 'metrostotales_agrupada'] = 200
trainProcesado.loc[((trainProcesado.metrostotales > 205) & (trainProcesado.metrostotales <= 255)), 'metrostotales_agrupada'] = 250
trainProcesado.loc[((trainProcesado.metrostotales > 255) & (trainProcesado.metrostotales <= 305)), 'metrostotales_agrupada'] = 300
trainProcesado.loc[((trainProcesado.metrostotales > 305) & (trainProcesado.metrostotales <= 355)), 'metrostotales_agrupada'] = 350
trainProcesado.loc[((trainProcesado.metrostotales > 355) & (trainProcesado.metrostotales <= 405)), 'metrostotales_agrupada'] = 400
trainProcesado.loc[((trainProcesado.metrostotales > 405) & (trainProcesado.metrostotales <= 455)), 'metrostotales_agrupada'] = 450
trainProcesado.loc[((trainProcesado.metrostotales > 455) & (trainProcesado.metrostotales <= 500)), 'metrostotales_agrupada'] = 500
trainProcesado.loc[((trainProcesado.metrostotales > 505) & (trainProcesado.metrostotales <= 555)), 'metrostotales_agrupada'] = 550
trainProcesado.loc[((trainProcesado.metrostotales > 555) & (trainProcesado.metrostotales <= 600)), 'metrostotales_agrupada'] = 600
trainProcesado.loc[((trainProcesado.metrostotales > 605) & (trainProcesado.metrostotales <= 655)), 'metrostotales_agrupada'] = 650
trainProcesado.loc[((trainProcesado.metrostotales > 655) & (trainProcesado.metrostotales <= 700)), 'metrostotales_agrupada'] = 700
trainProcesado.loc[((trainProcesado.metrostotales > 705) & (trainProcesado.metrostotales <= 755)), 'metrostotales_agrupada'] = 750
trainProcesado.loc[((trainProcesado.metrostotales > 755) & (trainProcesado.metrostotales <= 805)), 'metrostotales_agrupada'] = 800

trainProcesado.drop('metrostotales', 1, inplace = True)


In [42]:
train_gruouped = trainProcesado.groupby(['idzona','tipodepropiedad','habitaciones','anio'])['precio'].agg({'mean','median'}).reset_index()
trainProcesado = pd.merge(trainProcesado, train_gruouped, how='left', left_on=['idzona','tipodepropiedad','habitaciones','anio'], right_on = ['idzona','tipodepropiedad','habitaciones','anio'])

In [43]:
#Procesamiento de columna titulo
#Parseamos las palabras seleccionadas en el DF (ver notebook de exploracion)

#Nuevo dataframe
titleFeatures = pd.DataFrame()
titleFeatures['id'] = train['id']
#Uppercase
titleFeatures['titulo'] = train['titulo'].str.upper()
#Limpiamos las columnas
titleFeatures['titulo'] = titleFeatures['titulo'].str.normalize('NFKD')\
                                                .str.encode('ascii', errors='ignore')\
                                                .str.decode('utf-8')

#Armamos las columnas para cada palabra seleccionada
#Valor de inicializacion

#Propiedad nueva
titleFeatures['NUEVA'] = 0
#Calificativo (no aporta mucho)
titleFeatures['EXCELENTE'] = 0
#Tipo de aviso
titleFeatures['VENTA'] = 0
titleFeatures['PREVENTA'] = 0
titleFeatures['RENTA'] = 0
titleFeatures['CREDITO'] = 0
titleFeatures['REMATE'] = 0
titleFeatures['OPORTUNIDAD'] = 0
#Tipo de propiedad -extra
titleFeatures['RESIDENCIA'] = 0
titleFeatures['EDIFICIO'] = 0
titleFeatures['TORRE'] = 0
titleFeatures['PISO'] = 0
titleFeatures['LOFT'] = 0
titleFeatures['FRACCIONAMIENTO'] = 0
titleFeatures['DUPLEX'] = 0
titleFeatures['COUNTRY'] = 0
titleFeatures['QUINTA'] = 0
titleFeatures['OFICINA'] = 0
titleFeatures['PENTHOUSE'] = 0
#Tipo de propiedad -extra 2
titleFeatures['REMODELADO'] = 0
titleFeatures['MODERNO'] = 0
titleFeatures['MINIMALISTA'] = 0
titleFeatures['EQUIPADA'] = 0
#Jardin
titleFeatures['JARDIN'] = 0
#Ubicacion
titleFeatures['FRENTE'] = 0
titleFeatures['ESQUINA'] = 0
titleFeatures['CENTRO'] = 0
titleFeatures['UBICACION'] = 0
#Vistas o panoramicas
titleFeatures['VISTA'] = 0
#Caracteristicas extra
titleFeatures['LUJO'] = 0
titleFeatures['BODEGA'] = 0
titleFeatures['CAMPESTRE'] = 0
titleFeatures['RANCHO'] = 0
titleFeatures['PLAYA'] = 0
titleFeatures['GOLF'] = 0
#Seguridad
titleFeatures['SEGURIDAD'] = 0

#Buscamos y agregamos 1 donde aparece la palabra
titleFeatures.loc[titleFeatures['titulo'].str.contains('NUEVA', na=False), 'NUEVA'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('NUEVO', na=False), 'NUEVA'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('NUEVAS', na=False), 'NUEVA'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('ESTRENE', na=False), 'NUEVA'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('ESTRENAR', na=False), 'NUEVA'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('NUEVOS', na=False), 'NUEVA'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('ESTRENA', na=False), 'NUEVA'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('HERMOSA', na=False), 'EXCELENTE'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('EXELENTE', na=False), 'EXCELENTE'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('EXCELENTE', na=False), 'EXCELENTE'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('PRECIOSA', na=False), 'EXCELENTE'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('HERMOSO', na=False), 'EXCELENTE'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('BONITO', na=False), 'EXCELENTE'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('EXCELENTES', na=False), 'EXCELENTE'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('INMEJORABLE', na=False), 'EXCELENTE'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('AGRADABLE', na=False), 'EXCELENTE'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('LINDA', na=False), 'EXCELENTE'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('PRECIOSO', na=False), 'EXCELENTE'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('MAGNIFICA', na=False), 'EXCELENTE'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('IMPECABLE', na=False), 'EXCELENTE'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('HERMOSAS', na=False), 'EXCELENTE'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('EXCLUSIVA', na=False), 'EXCELENTE'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('ESPECTACULAR', na=False), 'EXCELENTE'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('INCREIBLE', na=False), 'EXCELENTE'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('EXCLUSIVO', na=False), 'EXCELENTE'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('SUPER', na=False), 'EXCELENTE'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('LINDO', na=False), 'EXCELENTE'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('MAGNIFICO', na=False), 'EXCELENTE'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('VENTA', na=False), 'VENTA'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('PREVENTA', na=False), 'PREVENTA'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('PRE-VENTA', na=False), 'PREVENTA'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('RENTA', na=False), 'RENTA'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('CREDITO', na=False), 'CREDITO'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('HIPOTECARIO', na=False), 'CREDITO'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('REMATE', na=False), 'REMATE'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('REMATO', na=False), 'REMATE'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('OPORTUNIDAD', na=False), 'OPORTUNIDAD'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('RESIDENCIA', na=False), 'RESIDENCIA'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('RESIDENCIAL', na=False), 'RESIDENCIA'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('EDIFICIO', na=False), 'EDIFICIO'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('TORRE', na=False), 'TORRE'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('PISO', na=False), 'PISO'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('LOFT', na=False), 'LOFT'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('FRACCIONAMIENTO', na=False), 'FRACCIONAMIENTO'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('FRACC', na=False), 'FRACCIONAMIENTO'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('DUPLEX', na=False), 'DUPLEX'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('COUNTRY', na=False), 'COUNTRY'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('QUINTA', na=False), 'QUINTA'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('OFICINA', na=False), 'OFICINA'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('PENTHOUSE', na=False), 'PENTHOUSE'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('REMODELADO', na=False), 'REMODELADO'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('REMODELADA', na=False), 'REMODELADO'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('MODERNA', na=False), 'MODERNO'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('MODERNO', na=False), 'MODERNO'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('MINIMALISTA', na=False), 'MINIMALISTA'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('EQUIPADA', na=False), 'EQUIPADA'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('JARDINES', na=False), 'JARDIN'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('JARDIN', na=False), 'JARDIN'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('VERDES', na=False), 'JARDIN'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('ARBOLEDAS', na=False), 'JARDIN'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('PARQUES', na=False), 'JARDIN'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('PATIO', na=False), 'JARDIN'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('FRENTE', na=False), 'FRENTE'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('ESQUINA', na=False), 'ESQUINA'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('CENTRO', na=False), 'CENTRO'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('CENTRAL', na=False), 'CENTRO'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('UBICACION', na=False), 'UBICACION'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('UBICADISIMO', na=False), 'UBICACION'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('PANORAMICA', na=False), 'VISTA'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('VISTA', na=False), 'VISTA'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('VISTAS', na=False), 'VISTA'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('LUJO', na=False), 'LUJO'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('BODEGA', na=False), 'BODEGA'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('CAMPESTRE', na=False), 'CAMPESTRE'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('RANCHO', na=False), 'RANCHO'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('PLAYA', na=False), 'PLAYA'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('GOLF', na=False), 'GOLF'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('SEGURIDAD', na=False), 'SEGURIDAD'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('VIGILANCIA', na=False), 'SEGURIDAD'] = 1

#Alternativamente, llenamos una sola columna como categorica, para luego encodear con LE
titleFeatures['titulo_features'] = 'NINGUNA'
titleFeatures.loc[titleFeatures['titulo'].str.contains('NUEVA', na=False), 'titulo_features'] = 'NUEVA'
titleFeatures.loc[titleFeatures['titulo'].str.contains('NUEVO', na=False), 'titulo_features'] = 'NUEVA'
titleFeatures.loc[titleFeatures['titulo'].str.contains('NUEVAS', na=False), 'titulo_features'] = 'NUEVA'
titleFeatures.loc[titleFeatures['titulo'].str.contains('ESTRENE', na=False), 'titulo_features'] = 'NUEVA'
titleFeatures.loc[titleFeatures['titulo'].str.contains('ESTRENAR', na=False), 'titulo_features'] = 'NUEVA'
titleFeatures.loc[titleFeatures['titulo'].str.contains('NUEVOS', na=False), 'titulo_features'] = 'NUEVA'
titleFeatures.loc[titleFeatures['titulo'].str.contains('ESTRENA', na=False), 'titulo_features'] = 'NUEVA'
titleFeatures.loc[titleFeatures['titulo'].str.contains('HERMOSA', na=False), 'titulo_features'] = 'EXCELENTE'
titleFeatures.loc[titleFeatures['titulo'].str.contains('EXELENTE', na=False), 'titulo_features'] = 'EXCELENTE'
titleFeatures.loc[titleFeatures['titulo'].str.contains('EXCELENTE', na=False), 'titulo_features'] = 'EXCELENTE'
titleFeatures.loc[titleFeatures['titulo'].str.contains('PRECIOSA', na=False), 'titulo_features'] = 'EXCELENTE'
titleFeatures.loc[titleFeatures['titulo'].str.contains('HERMOSO', na=False), 'titulo_features'] = 'EXCELENTE'
titleFeatures.loc[titleFeatures['titulo'].str.contains('BONITO', na=False), 'titulo_features'] = 'EXCELENTE'
titleFeatures.loc[titleFeatures['titulo'].str.contains('EXCELENTES', na=False), 'titulo_features'] = 'EXCELENTE'
titleFeatures.loc[titleFeatures['titulo'].str.contains('INMEJORABLE', na=False), 'titulo_features'] = 'EXCELENTE'
titleFeatures.loc[titleFeatures['titulo'].str.contains('AGRADABLE', na=False), 'titulo_features'] = 'EXCELENTE'
titleFeatures.loc[titleFeatures['titulo'].str.contains('LINDA', na=False), 'titulo_features'] = 'EXCELENTE'
titleFeatures.loc[titleFeatures['titulo'].str.contains('PRECIOSO', na=False), 'titulo_features'] = 'EXCELENTE'
titleFeatures.loc[titleFeatures['titulo'].str.contains('MAGNIFICA', na=False), 'titulo_features'] = 'EXCELENTE'
titleFeatures.loc[titleFeatures['titulo'].str.contains('IMPECABLE', na=False), 'titulo_features'] = 'EXCELENTE'
titleFeatures.loc[titleFeatures['titulo'].str.contains('HERMOSAS', na=False), 'titulo_features'] = 'EXCELENTE'
titleFeatures.loc[titleFeatures['titulo'].str.contains('EXCLUSIVA', na=False), 'titulo_features'] = 'EXCELENTE'
titleFeatures.loc[titleFeatures['titulo'].str.contains('ESPECTACULAR', na=False), 'titulo_features'] = 'EXCELENTE'
titleFeatures.loc[titleFeatures['titulo'].str.contains('INCREIBLE', na=False), 'titulo_features'] = 'EXCELENTE'
titleFeatures.loc[titleFeatures['titulo'].str.contains('EXCLUSIVO', na=False), 'titulo_features'] = 'EXCELENTE'
titleFeatures.loc[titleFeatures['titulo'].str.contains('SUPER', na=False), 'titulo_features'] = 'EXCELENTE'
titleFeatures.loc[titleFeatures['titulo'].str.contains('LINDO', na=False), 'titulo_features'] = 'EXCELENTE'
titleFeatures.loc[titleFeatures['titulo'].str.contains('MAGNIFICO', na=False), 'titulo_features'] = 'EXCELENTE'
titleFeatures.loc[titleFeatures['titulo'].str.contains('VENTA', na=False), 'titulo_features'] = 'VENTA'
titleFeatures.loc[titleFeatures['titulo'].str.contains('PREVENTA', na=False), 'titulo_features'] = 'PREVENTA'
titleFeatures.loc[titleFeatures['titulo'].str.contains('PRE-VENTA', na=False), 'titulo_features'] = 'PREVENTA'
titleFeatures.loc[titleFeatures['titulo'].str.contains('RENTA', na=False), 'titulo_features'] = 'RENTA'
titleFeatures.loc[titleFeatures['titulo'].str.contains('CREDITO', na=False), 'titulo_features'] = 'CREDITO'
titleFeatures.loc[titleFeatures['titulo'].str.contains('HIPOTECARIO', na=False), 'titulo_features'] = 'CREDITO'
titleFeatures.loc[titleFeatures['titulo'].str.contains('REMATE', na=False), 'titulo_features'] = 'REMATE'
titleFeatures.loc[titleFeatures['titulo'].str.contains('REMATO', na=False), 'titulo_features'] = 'REMATE'
titleFeatures.loc[titleFeatures['titulo'].str.contains('OPORTUNIDAD', na=False), 'titulo_features'] = 'OPORTUNIDAD'
titleFeatures.loc[titleFeatures['titulo'].str.contains('RESIDENCIA', na=False), 'titulo_features'] = 'RESIDENCIA'
titleFeatures.loc[titleFeatures['titulo'].str.contains('RESIDENCIAL', na=False), 'titulo_features'] = 'RESIDENCIA'
titleFeatures.loc[titleFeatures['titulo'].str.contains('EDIFICIO', na=False), 'titulo_features'] = 'EDIFICIO'
titleFeatures.loc[titleFeatures['titulo'].str.contains('TORRE', na=False), 'titulo_features'] = 'TORRE'
titleFeatures.loc[titleFeatures['titulo'].str.contains('PISO', na=False), 'titulo_features'] = 'PISO'
titleFeatures.loc[titleFeatures['titulo'].str.contains('LOFT', na=False), 'titulo_features'] = 'LOFT'
titleFeatures.loc[titleFeatures['titulo'].str.contains('FRACCIONAMIENTO', na=False), 'titulo_features'] = 'FRACCIONAMIENTO'
titleFeatures.loc[titleFeatures['titulo'].str.contains('FRACC', na=False), 'titulo_features'] = 'FRACCIONAMIENTO'
titleFeatures.loc[titleFeatures['titulo'].str.contains('DUPLEX', na=False), 'titulo_features'] = 'DUPLEX'
titleFeatures.loc[titleFeatures['titulo'].str.contains('COUNTRY', na=False), 'titulo_features'] = 'COUNTRY'
titleFeatures.loc[titleFeatures['titulo'].str.contains('QUINTA', na=False), 'titulo_features'] = 'QUINTA'
titleFeatures.loc[titleFeatures['titulo'].str.contains('OFICINA', na=False), 'titulo_features'] = 'OFICINA'
titleFeatures.loc[titleFeatures['titulo'].str.contains('PENTHOUSE', na=False), 'titulo_features'] = 'PENTHOUSE'
titleFeatures.loc[titleFeatures['titulo'].str.contains('REMODELADO', na=False), 'titulo_features'] = 'REMODELADO'
titleFeatures.loc[titleFeatures['titulo'].str.contains('REMODELADA', na=False), 'titulo_features'] = 'REMODELADO'
titleFeatures.loc[titleFeatures['titulo'].str.contains('MODERNA', na=False), 'titulo_features'] = 'MODERNO'
titleFeatures.loc[titleFeatures['titulo'].str.contains('MODERNO', na=False), 'titulo_features'] = 'MODERNO'
titleFeatures.loc[titleFeatures['titulo'].str.contains('MINIMALISTA', na=False), 'titulo_features'] = 'MINIMALISTA'
titleFeatures.loc[titleFeatures['titulo'].str.contains('EQUIPADA', na=False), 'titulo_features'] = 'EQUIPADA'
titleFeatures.loc[titleFeatures['titulo'].str.contains('JARDINES', na=False), 'titulo_features'] = 'JARDIN'
titleFeatures.loc[titleFeatures['titulo'].str.contains('JARDIN', na=False), 'titulo_features'] = 'JARDIN'
titleFeatures.loc[titleFeatures['titulo'].str.contains('VERDES', na=False), 'titulo_features'] = 'JARDIN'
titleFeatures.loc[titleFeatures['titulo'].str.contains('ARBOLEDAS', na=False), 'titulo_features'] = 'JARDIN'
titleFeatures.loc[titleFeatures['titulo'].str.contains('PARQUES', na=False), 'titulo_features'] = 'JARDIN'
titleFeatures.loc[titleFeatures['titulo'].str.contains('PATIO', na=False), 'titulo_features'] = 'JARDIN'
titleFeatures.loc[titleFeatures['titulo'].str.contains('FRENTE', na=False), 'titulo_features'] = 'FRENTE'
titleFeatures.loc[titleFeatures['titulo'].str.contains('ESQUINA', na=False), 'titulo_features'] = 'ESQUINA'
titleFeatures.loc[titleFeatures['titulo'].str.contains('CENTRO', na=False), 'titulo_features'] = 'CENTRO'
titleFeatures.loc[titleFeatures['titulo'].str.contains('CENTRAL', na=False), 'titulo_features'] = 'CENTRO'
titleFeatures.loc[titleFeatures['titulo'].str.contains('UBICACION', na=False), 'titulo_features'] = 'UBICACION'
titleFeatures.loc[titleFeatures['titulo'].str.contains('UBICADISIMO', na=False), 'titulo_features'] = 'UBICACION'
titleFeatures.loc[titleFeatures['titulo'].str.contains('PANORAMICA', na=False), 'titulo_features'] = 'VISTA'
titleFeatures.loc[titleFeatures['titulo'].str.contains('VISTA', na=False), 'titulo_features'] = 'VISTA'
titleFeatures.loc[titleFeatures['titulo'].str.contains('VISTAS', na=False), 'titulo_features'] = 'VISTA'
titleFeatures.loc[titleFeatures['titulo'].str.contains('LUJO', na=False), 'titulo_features'] = 'LUJO'
titleFeatures.loc[titleFeatures['titulo'].str.contains('BODEGA', na=False), 'titulo_features'] = 'BODEGA'
titleFeatures.loc[titleFeatures['titulo'].str.contains('CAMPESTRE', na=False), 'titulo_features'] = 'CAMPESTRE'
titleFeatures.loc[titleFeatures['titulo'].str.contains('RANCHO', na=False), 'titulo_features'] = 'RANCHO'
titleFeatures.loc[titleFeatures['titulo'].str.contains('PLAYA', na=False), 'titulo_features'] = 'PLAYA'
titleFeatures.loc[titleFeatures['titulo'].str.contains('GOLF', na=False), 'titulo_features'] = 'GOLF'
titleFeatures.loc[titleFeatures['titulo'].str.contains('SEGURIDAD', na=False), 'titulo_features'] = 'SEGURIDAD'
titleFeatures.loc[titleFeatures['titulo'].str.contains('VIGILANCIA', na=False), 'titulo_features'] = 'SEGURIDAD'
#La tipamos
titleFeatures['titulo_features'] = titleFeatures['titulo_features'].astype('category')

#La guardamos con label encoding
cat_features_titulo = ['titulo_features']
encoder_titulo = LabelEncoder()
encoded_titulo = titleFeatures[cat_features_titulo].apply(encoder_titulo.fit_transform)
encoded_titulo['id'] = train['id']
encoded_titulo = encoded_titulo.rename(columns={'titulo_features': 'titulo_features_enc'})

#Agregamos la columna al los DF
titleFeatures = pd.merge(titleFeatures,encoded_titulo, on='id', how='inner')
trainProcesado = pd.merge(trainProcesado,titleFeatures, on='id', how='inner')
trainProcesado = trainProcesado.drop('titulo',1)
trainProcesado = trainProcesado.drop('titulo_features',1)
trainProcesado = trainProcesado.drop('titulo_features_enc',1)

In [44]:
#Tratamiento analogo para la columna descripcion

#Procesamiento de columna descripcion
#Parseamos las palabras seleccionadas en el DF (ver notebook de exploracion)

#Nuevo dataframe
descFeatures = pd.DataFrame()
descFeatures['id'] = train['id']
#Uppercase
descFeatures['descripcion'] = train['descripcion'].str.upper()
#Limpiamos las columnas
descFeatures['descripcion'] = descFeatures['descripcion'].str.normalize('NFKD')\
                                                .str.encode('ascii', errors='ignore')\
                                                .str.decode('utf-8')

#Armamos las columnas para cada palabra seleccionada
#Valor de inicializacion

#Ya usadas en titulo
#Propiedad nueva
descFeatures['NUEVA'] = 0
#Calificativo (no aporta mucho)
descFeatures['EXCELENTE'] = 0
#Tipo de aviso
descFeatures['VENTA'] = 0
descFeatures['PREVENTA'] = 0
descFeatures['RENTA'] = 0
descFeatures['CREDITO'] = 0
descFeatures['REMATE'] = 0
descFeatures['OPORTUNIDAD'] = 0
#Tipo de propiedad -extra
descFeatures['RESIDENCIA'] = 0
descFeatures['EDIFICIO'] = 0
descFeatures['TORRE'] = 0
descFeatures['PISO'] = 0
descFeatures['LOFT'] = 0
descFeatures['FRACCIONAMIENTO'] = 0
descFeatures['DUPLEX'] = 0
descFeatures['COUNTRY'] = 0
descFeatures['QUINTA'] = 0
descFeatures['OFICINA'] = 0
descFeatures['PENTHOUSE'] = 0
#Tipo de propiedad -extra 2
descFeatures['REMODELADO'] = 0
descFeatures['MODERNO'] = 0
descFeatures['MINIMALISTA'] = 0
descFeatures['EQUIPADA'] = 0
#Jardin
descFeatures['JARDIN'] = 0
#Ubicacion
descFeatures['FRENTE'] = 0
descFeatures['ESQUINA'] = 0
descFeatures['CENTRO'] = 0
descFeatures['UBICACION'] = 0
#Vistas o panoramicas
descFeatures['VISTA'] = 0
#Caracteristicas extra
descFeatures['LUJO'] = 0
descFeatures['BODEGA'] = 0
descFeatures['CAMPESTRE'] = 0
descFeatures['RANCHO'] = 0
descFeatures['PLAYA'] = 0
descFeatures['GOLF'] = 0
#Seguridad
descFeatures['SEGURIDAD'] = 0

#Nuevas de descripcion
#Iluminacion
descFeatures['ILUMINADA'] = 0
#Ubicacion
descFeatures['TRANQUILO'] = 0
descFeatures['UNIVERSIDAD'] = 0
descFeatures['AVENIDA'] = 0
#Caracteristicas extra
descFeatures['AMENIDADES'] = 0
descFeatures['BARRA'] = 0
descFeatures['BALCON'] = 0
descFeatures['RECIBIDOR'] = 0
descFeatures['AMENIDADES'] = 0
descFeatures['DESPENSA'] = 0
descFeatures['LAVADERO'] = 0
descFeatures['REJA'] = 0
descFeatures['TERRAZA'] = 0
descFeatures['CHIMENEA'] = 0
descFeatures['CISTERNA'] = 0

#Buscamos y agregamos 1 donde aparece la palabra
descFeatures.loc[descFeatures['descripcion'].str.contains('NUEVO', na=False), 'NUEVA'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('ESTRENAR', na=False), 'NUEVA'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('EXCELENTE', na=False), 'EXCELENTE'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('HERMOSA', na=False), 'EXCELENTE'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('BONITA', na=False), 'EXCELENTE'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('PRECIOSA', na=False), 'EXCELENTE'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('HERMOSO', na=False), 'EXCELENTE'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('AGRADABLE', na=False), 'EXCELENTE'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('PRECIOSO', na=False), 'EXCELENTE'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('ESPECTACULAR', na=False), 'EXCELENTE'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('HERMOSAS', na=False), 'EXCELENTE'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('HERMOSOS', na=False), 'EXCELENTE'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('BONITOS', na=False), 'EXCELENTE'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('INMEJORABLE', na=False), 'EXCELENTE'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('REMATES', na=False), 'REMATE'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('HIPOTECARIO', na=False), 'CREDITO'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('INVERSION', na=False), 'OPORTUNIDAD'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('INVERSIONISTAS', na=False), 'OPORTUNIDAD'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('RESIDENCIAL', na=False), 'RESIDENCIA'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('FRACCIONAMIENTO', na=False), 'FRACCIONAMIENTO'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('ESTANCIA', na=False), 'RANCHO'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('PISO', na=False), 'PISO'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('TORRE', na=False), 'TORRE'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('LUZ', na=False), 'ILUMINADA'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('ILUMINADA', na=False), 'ILUMINADA'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('ILUMINADO', na=False), 'ILUMINADA'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('ILUMINACION', na=False), 'ILUMINADA'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('PATIO', na=False), 'JARDIN'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('FONDO', na=False), 'JARDIN'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('JARDIN', na=False), 'JARDIN'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('VERDES', na=False), 'JARDIN'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('VERDE', na=False), 'JARDIN'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('GARDEN', na=False), 'JARDIN'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('PARQUE', na=False), 'JARDIN'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('NATURAL', na=False), 'JARDIN'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('BOSQUES', na=False), 'JARDIN'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('PARQUES', na=False), 'JARDIN'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('ARBOLES', na=False), 'JARDIN'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('VIGILANCIA', na=False), 'SEGURIDAD'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('SEGURIDAD', na=False), 'SEGURIDAD'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('SEGURO', na=False), 'SEGURIDAD'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('VISTAS', na=False), 'VISTA'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('PANORAMICA', na=False), 'VISTA'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('VISTA', na=False), 'VISTA'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('UBICACION', na=False), 'UBICACION'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('UBICADO', na=False), 'UBICACION'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('TRANQUILA', na=False), 'TRANQUILO'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('TRANQUILO', na=False), 'TRANQUILO'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('TRANQUILIDAD', na=False), 'TRANQUILO'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('UNIVERSIDAD', na=False), 'UNIVERSIDAD'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('AVENIDA', na=False), 'AVENIDA'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('ESQUINA', na=False), 'ESQUINA'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('FACHADA', na=False), 'FRENTE'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('FRENTE', na=False), 'FRENTE'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('REMODELADO', na=False), 'REMODELADO'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('REMODELADA', na=False), 'REMODELADO'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('MODERNA', na=False), 'MODERNO'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('EXCLUSIVA', na=False), 'LUJO'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('LUJO', na=False), 'LUJO'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('CALIDAD', na=False), 'LUJO'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('MARMOL', na=False), 'LUJO'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('PORCELANATO', na=False), 'LUJO'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('JACUZZI', na=False), 'LUJO'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('VESTIDOR', na=False), 'LUJO'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('GOLF', na=False), 'GOLF'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('BODEGA', na=False), 'BODEGA'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('PLAYA', na=False), 'PLAYA'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('AMENIDADES ', na=False), 'AMENIDADES'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('FIESTAS', na=False), 'AMENIDADES'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('EVENTOS', na=False), 'AMENIDADES'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('LAVANDERIA', na=False), 'AMENIDADES'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('JUEGOS', na=False), 'AMENIDADES'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('SERVICIOS', na=False), 'AMENIDADES'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('SERVICIO', na=False), 'AMENIDADES'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('ASADOR', na=False), 'AMENIDADES'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('CANCHAS', na=False), 'AMENIDADES'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('TENIS', na=False), 'AMENIDADES'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('BARRA', na=False), 'BARRA'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('DESAYUNADORA', na=False), 'BARRA'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('DESAYUNADOR', na=False), 'BARRA'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('BALCON', na=False), 'BALCON'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('RECIBIDOR', na=False), 'RECIBIDOR'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('VESTIBULO', na=False), 'RECIBIDOR'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('CHIMENEA', na=False), 'CHIMENEA'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('DESPENSA', na=False), 'DESPENSA'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('LAVADERO', na=False), 'LAVADERO'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('REJA', na=False), 'REJA'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('BOILER', na=False), 'CISTERNA'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('CISTERNA', na=False), 'CISTERNA'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('AZOTEA', na=False), 'TERRAZA'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('TERRAZA', na=False), 'TERRAZA'] = 1

#Alternativamente, llenamos una sola columna como categorica, para luego encodear con LE
descFeatures['descripcion_features'] = 'NINGUNA'
descFeatures.loc[descFeatures['descripcion'].str.contains('NUEVO', na=False), 'descripcion_features'] = 'NUEVA'
descFeatures.loc[descFeatures['descripcion'].str.contains('ESTRENAR', na=False), 'descripcion_features'] = 'NUEVA'
descFeatures.loc[descFeatures['descripcion'].str.contains('EXCELENTE', na=False), 'descripcion_features'] = 'EXCELENTE'
descFeatures.loc[descFeatures['descripcion'].str.contains('HERMOSA', na=False), 'descripcion_features'] = 'EXCELENTE'
descFeatures.loc[descFeatures['descripcion'].str.contains('BONITA', na=False), 'descripcion_features'] = 'EXCELENTE'
descFeatures.loc[descFeatures['descripcion'].str.contains('PRECIOSA', na=False), 'descripcion_features'] = 'EXCELENTE'
descFeatures.loc[descFeatures['descripcion'].str.contains('HERMOSO', na=False), 'descripcion_features'] = 'EXCELENTE'
descFeatures.loc[descFeatures['descripcion'].str.contains('AGRADABLE', na=False), 'descripcion_features'] = 'EXCELENTE'
descFeatures.loc[descFeatures['descripcion'].str.contains('PRECIOSO', na=False), 'descripcion_features'] = 'EXCELENTE'
descFeatures.loc[descFeatures['descripcion'].str.contains('ESPECTACULAR', na=False), 'descripcion_features'] = 'EXCELENTE'
descFeatures.loc[descFeatures['descripcion'].str.contains('HERMOSAS', na=False), 'descripcion_features'] = 'EXCELENTE'
descFeatures.loc[descFeatures['descripcion'].str.contains('HERMOSOS', na=False), 'descripcion_features'] = 'EXCELENTE'
descFeatures.loc[descFeatures['descripcion'].str.contains('BONITOS', na=False), 'descripcion_features'] = 'EXCELENTE'
descFeatures.loc[descFeatures['descripcion'].str.contains('INMEJORABLE', na=False), 'descripcion_features'] = 'EXCELENTE'
descFeatures.loc[descFeatures['descripcion'].str.contains('REMATES', na=False), 'descripcion_features'] = 'REMATE'
descFeatures.loc[descFeatures['descripcion'].str.contains('HIPOTECARIO', na=False), 'descripcion_features'] = 'CREDITO'
descFeatures.loc[descFeatures['descripcion'].str.contains('INVERSION', na=False), 'descripcion_features'] = 'OPORTUNIDAD'
descFeatures.loc[descFeatures['descripcion'].str.contains('INVERSIONISTAS', na=False), 'descripcion_features'] = 'OPORTUNIDAD'
descFeatures.loc[descFeatures['descripcion'].str.contains('RESIDENCIAL', na=False), 'descripcion_features'] = 'RESIDENCIA'
descFeatures.loc[descFeatures['descripcion'].str.contains('FRACCIONAMIENTO', na=False), 'descripcion_features'] = 'FRACCIONAMIENTO'
descFeatures.loc[descFeatures['descripcion'].str.contains('ESTANCIA', na=False), 'descripcion_features'] = 'RANCHO'
descFeatures.loc[descFeatures['descripcion'].str.contains('PISO', na=False), 'descripcion_features'] = 'PISO'
descFeatures.loc[descFeatures['descripcion'].str.contains('TORRE', na=False), 'descripcion_features'] = 'TORRE'
descFeatures.loc[descFeatures['descripcion'].str.contains('LUZ', na=False), 'descripcion_features'] = 'ILUMINADA'
descFeatures.loc[descFeatures['descripcion'].str.contains('ILUMINADA', na=False), 'descripcion_features'] = 'ILUMINADA'
descFeatures.loc[descFeatures['descripcion'].str.contains('ILUMINADO', na=False), 'descripcion_features'] = 'ILUMINADA'
descFeatures.loc[descFeatures['descripcion'].str.contains('ILUMINACION', na=False), 'descripcion_features'] = 'ILUMINADA'
descFeatures.loc[descFeatures['descripcion'].str.contains('PATIO', na=False), 'descripcion_features'] = 'JARDIN'
descFeatures.loc[descFeatures['descripcion'].str.contains('FONDO', na=False), 'descripcion_features'] = 'JARDIN'
descFeatures.loc[descFeatures['descripcion'].str.contains('JARDIN', na=False), 'descripcion_features'] = 'JARDIN'
descFeatures.loc[descFeatures['descripcion'].str.contains('VERDES', na=False), 'descripcion_features'] = 'JARDIN'
descFeatures.loc[descFeatures['descripcion'].str.contains('VERDE', na=False), 'descripcion_features'] = 'JARDIN'
descFeatures.loc[descFeatures['descripcion'].str.contains('GARDEN', na=False), 'descripcion_features'] = 'JARDIN'
descFeatures.loc[descFeatures['descripcion'].str.contains('PARQUE', na=False), 'descripcion_features'] = 'JARDIN'
descFeatures.loc[descFeatures['descripcion'].str.contains('NATURAL', na=False), 'descripcion_features'] = 'JARDIN'
descFeatures.loc[descFeatures['descripcion'].str.contains('BOSQUES', na=False), 'descripcion_features'] = 'JARDIN'
descFeatures.loc[descFeatures['descripcion'].str.contains('PARQUES', na=False), 'descripcion_features'] = 'JARDIN'
descFeatures.loc[descFeatures['descripcion'].str.contains('ARBOLES', na=False), 'descripcion_features'] = 'JARDIN'
descFeatures.loc[descFeatures['descripcion'].str.contains('VIGILANCIA', na=False), 'descripcion_features'] = 'SEGURIDAD'
descFeatures.loc[descFeatures['descripcion'].str.contains('SEGURIDAD', na=False), 'descripcion_features'] = 'SEGURIDAD'
descFeatures.loc[descFeatures['descripcion'].str.contains('SEGURO', na=False), 'descripcion_features'] = 'SEGURIDAD'
descFeatures.loc[descFeatures['descripcion'].str.contains('VISTAS', na=False), 'descripcion_features'] = 'VISTA'
descFeatures.loc[descFeatures['descripcion'].str.contains('PANORAMICA', na=False), 'descripcion_features'] = 'VISTA'
descFeatures.loc[descFeatures['descripcion'].str.contains('VISTA', na=False), 'descripcion_features'] = 'VISTA'
descFeatures.loc[descFeatures['descripcion'].str.contains('UBICACION', na=False), 'descripcion_features'] = 'UBICACION'
descFeatures.loc[descFeatures['descripcion'].str.contains('UBICADO', na=False), 'descripcion_features'] = 'UBICACION'
descFeatures.loc[descFeatures['descripcion'].str.contains('TRANQUILA', na=False), 'descripcion_features'] = 'TRANQUILO'
descFeatures.loc[descFeatures['descripcion'].str.contains('TRANQUILO', na=False), 'descripcion_features'] = 'TRANQUILO'
descFeatures.loc[descFeatures['descripcion'].str.contains('TRANQUILIDAD', na=False), 'descripcion_features'] = 'TRANQUILO'
descFeatures.loc[descFeatures['descripcion'].str.contains('UNIVERSIDAD', na=False), 'descripcion_features'] = 'UNIVERSIDAD'
descFeatures.loc[descFeatures['descripcion'].str.contains('AVENIDA', na=False), 'descripcion_features'] = 'AVENIDA'
descFeatures.loc[descFeatures['descripcion'].str.contains('ESQUINA', na=False), 'descripcion_features'] = 'ESQUINA'
descFeatures.loc[descFeatures['descripcion'].str.contains('FACHADA', na=False), 'descripcion_features'] = 'FRENTE'
descFeatures.loc[descFeatures['descripcion'].str.contains('FRENTE', na=False), 'descripcion_features'] = 'FRENTE'
descFeatures.loc[descFeatures['descripcion'].str.contains('REMODELADO', na=False), 'descripcion_features'] = 'REMODELADO'
descFeatures.loc[descFeatures['descripcion'].str.contains('REMODELADA', na=False), 'descripcion_features'] = 'REMODELADO'
descFeatures.loc[descFeatures['descripcion'].str.contains('MODERNA', na=False), 'descripcion_features'] = 'MODERNO'
descFeatures.loc[descFeatures['descripcion'].str.contains('EXCLUSIVA', na=False), 'descripcion_features'] = 'LUJO'
descFeatures.loc[descFeatures['descripcion'].str.contains('LUJO', na=False), 'descripcion_features'] = 'LUJO'
descFeatures.loc[descFeatures['descripcion'].str.contains('CALIDAD', na=False), 'descripcion_features'] = 'LUJO'
descFeatures.loc[descFeatures['descripcion'].str.contains('MARMOL', na=False), 'descripcion_features'] = 'LUJO'
descFeatures.loc[descFeatures['descripcion'].str.contains('PORCELANATO', na=False), 'descripcion_features'] = 'LUJO'
descFeatures.loc[descFeatures['descripcion'].str.contains('JACUZZI', na=False), 'descripcion_features'] = 'LUJO'
descFeatures.loc[descFeatures['descripcion'].str.contains('VESTIDOR', na=False), 'descripcion_features'] = 'LUJO'
descFeatures.loc[descFeatures['descripcion'].str.contains('GOLF', na=False), 'descripcion_features'] = 'GOLF'
descFeatures.loc[descFeatures['descripcion'].str.contains('BODEGA', na=False), 'descripcion_features'] = 'BODEGA'
descFeatures.loc[descFeatures['descripcion'].str.contains('PLAYA', na=False), 'descripcion_features'] = 'PLAYA'
descFeatures.loc[descFeatures['descripcion'].str.contains('AMENIDADES ', na=False), 'descripcion_features'] = 'AMENIDADES'
descFeatures.loc[descFeatures['descripcion'].str.contains('FIESTAS', na=False), 'descripcion_features'] = 'AMENIDADES'
descFeatures.loc[descFeatures['descripcion'].str.contains('EVENTOS', na=False), 'descripcion_features'] = 'AMENIDADES'
descFeatures.loc[descFeatures['descripcion'].str.contains('LAVANDERIA', na=False), 'descripcion_features'] = 'AMENIDADES'
descFeatures.loc[descFeatures['descripcion'].str.contains('JUEGOS', na=False), 'descripcion_features'] = 'AMENIDADES'
descFeatures.loc[descFeatures['descripcion'].str.contains('SERVICIOS', na=False), 'descripcion_features'] = 'AMENIDADES'
descFeatures.loc[descFeatures['descripcion'].str.contains('SERVICIO', na=False), 'descripcion_features'] = 'AMENIDADES'
descFeatures.loc[descFeatures['descripcion'].str.contains('ASADOR', na=False), 'descripcion_features'] = 'AMENIDADES'
descFeatures.loc[descFeatures['descripcion'].str.contains('CANCHAS', na=False), 'descripcion_features'] = 'AMENIDADES'
descFeatures.loc[descFeatures['descripcion'].str.contains('TENIS', na=False), 'descripcion_features'] = 'AMENIDADES'
descFeatures.loc[descFeatures['descripcion'].str.contains('BARRA', na=False), 'descripcion_features'] = 'BARRA'
descFeatures.loc[descFeatures['descripcion'].str.contains('DESAYUNADORA', na=False), 'descripcion_features'] = 'BARRA'
descFeatures.loc[descFeatures['descripcion'].str.contains('DESAYUNADOR', na=False), 'descripcion_features'] = 'BARRA'
descFeatures.loc[descFeatures['descripcion'].str.contains('BALCON', na=False), 'descripcion_features'] = 'BALCON'
descFeatures.loc[descFeatures['descripcion'].str.contains('RECIBIDOR', na=False), 'descripcion_features'] = 'RECIBIDOR'
descFeatures.loc[descFeatures['descripcion'].str.contains('VESTIBULO', na=False), 'descripcion_features'] = 'RECIBIDOR'
descFeatures.loc[descFeatures['descripcion'].str.contains('CHIMENEA', na=False), 'descripcion_features'] = 'CHIMENEA'
descFeatures.loc[descFeatures['descripcion'].str.contains('DESPENSA', na=False), 'descripcion_features'] = 'DESPENSA'
descFeatures.loc[descFeatures['descripcion'].str.contains('LAVADERO', na=False), 'descripcion_features'] = 'LAVADERO'
descFeatures.loc[descFeatures['descripcion'].str.contains('REJA', na=False), 'descripcion_features'] = 'REJA'
descFeatures.loc[descFeatures['descripcion'].str.contains('BOILER', na=False), 'descripcion_features'] = 'CISTERNA'
descFeatures.loc[descFeatures['descripcion'].str.contains('CISTERNA', na=False), 'descripcion_features'] = 'CISTERNA'
descFeatures.loc[descFeatures['descripcion'].str.contains('AZOTEA', na=False), 'descripcion_features'] = 'TERRAZA'
descFeatures.loc[descFeatures['descripcion'].str.contains('TERRAZA', na=False), 'descripcion_features'] = 'TERRAZA'

#La tipamos
descFeatures['descripcion_features'] = descFeatures['descripcion_features'].astype('category')

#La guardamos con label encoding
cat_features_desc = ['descripcion_features']
encoder_desc = LabelEncoder()
encoded_desc = descFeatures[cat_features_desc].apply(encoder_desc.fit_transform)
encoded_desc['id'] = train['id']
encoded_desc = encoded_desc.rename(columns={'descripcion_features': 'descripcion_features_enc'})

#Agregamos la columna al los DF
descFeatures = pd.merge(descFeatures,encoded_desc, on='id', how='inner')
trainProcesado = pd.merge(trainProcesado,descFeatures, on='id', how='inner')
trainProcesado = trainProcesado.drop('descripcion',1)
trainProcesado = trainProcesado.drop('descripcion_features',1)
trainProcesado = trainProcesado.drop('descripcion_features_enc',1)

In [45]:
#OR entre las columnas repetidas en ambos DF
#Propiedad nueva
trainProcesado['NUEVA'] = trainProcesado['NUEVA_x'] | trainProcesado['NUEVA_y']
#Calificativo (no aporta mucho)
trainProcesado['EXCELENTE'] = trainProcesado['EXCELENTE_x'] | trainProcesado['EXCELENTE_y']
#Tipo de aviso
trainProcesado['VENTA'] = trainProcesado['VENTA_x'] | trainProcesado['VENTA_y'] 
trainProcesado['PREVENTA'] = trainProcesado['PREVENTA_x'] | trainProcesado['PREVENTA_y']
trainProcesado['RENTA'] = trainProcesado['RENTA_x'] | trainProcesado['RENTA_y'] 
trainProcesado['CREDITO'] = trainProcesado['CREDITO_x'] | trainProcesado['CREDITO_y']
trainProcesado['REMATE'] = trainProcesado['REMATE_x'] | trainProcesado['REMATE_y']
trainProcesado['OPORTUNIDAD'] = trainProcesado['OPORTUNIDAD_x'] | trainProcesado['OPORTUNIDAD_y']
#Tipo de propiedad -extra
trainProcesado['RESIDENCIA'] = trainProcesado['RESIDENCIA_x'] | trainProcesado['RESIDENCIA_y']
trainProcesado['EDIFICIO'] = trainProcesado['EDIFICIO_x'] | trainProcesado['EDIFICIO_y'] 
trainProcesado['TORRE'] = trainProcesado['TORRE_x'] | trainProcesado['TORRE_y']
trainProcesado['PISO'] = trainProcesado['PISO_x'] | trainProcesado['PISO_y']
trainProcesado['LOFT'] = trainProcesado['LOFT_x'] | trainProcesado['LOFT_y']
trainProcesado['FRACCIONAMIENTO'] = trainProcesado['FRACCIONAMIENTO_x'] | trainProcesado['FRACCIONAMIENTO_y']
trainProcesado['DUPLEX'] = trainProcesado['DUPLEX_x'] | trainProcesado['DUPLEX_y'] 
trainProcesado['COUNTRY'] = trainProcesado['COUNTRY_x'] | trainProcesado['COUNTRY_y']
trainProcesado['QUINTA'] = trainProcesado['QUINTA_x'] | trainProcesado['QUINTA_y']
trainProcesado['OFICINA'] = trainProcesado['OFICINA_x'] | trainProcesado['OFICINA_y']
trainProcesado['PENTHOUSE'] = trainProcesado['PENTHOUSE_x'] | trainProcesado['PENTHOUSE_y']
#Tipo de propiedad -extra 2
trainProcesado['REMODELADO'] = trainProcesado['REMODELADO_x'] | trainProcesado['REMODELADO_y']
trainProcesado['MODERNO'] = trainProcesado['MODERNO_x'] | trainProcesado['MODERNO_y']
trainProcesado['MINIMALISTA'] = trainProcesado['MINIMALISTA_x'] | trainProcesado['MINIMALISTA_y']
trainProcesado['EQUIPADA'] = trainProcesado['EQUIPADA_x'] | trainProcesado['EQUIPADA_y']
#Jardin
trainProcesado['JARDIN'] = trainProcesado['JARDIN_x'] | trainProcesado['JARDIN_y'] 
#Ubicacion
trainProcesado['FRENTE'] = trainProcesado['FRENTE_x'] | trainProcesado['FRENTE_y'] 
trainProcesado['ESQUINA'] = trainProcesado['ESQUINA_x'] | trainProcesado['ESQUINA_y'] 
trainProcesado['CENTRO'] = trainProcesado['CENTRO_x'] | trainProcesado['CENTRO_y']
trainProcesado['UBICACION'] = trainProcesado['UBICACION_x'] | trainProcesado['UBICACION_y']
#Vistas o panoramicas
trainProcesado['VISTA'] = trainProcesado['VISTA_x'] | trainProcesado['VISTA_y']
#Caracteristicas extra
trainProcesado['LUJO'] = trainProcesado['LUJO_x'] |  trainProcesado['LUJO_y']
trainProcesado['BODEGA'] = trainProcesado['BODEGA_x'] | trainProcesado['BODEGA_y']
trainProcesado['CAMPESTRE'] = trainProcesado['CAMPESTRE_x'] | trainProcesado['CAMPESTRE_x']
trainProcesado['RANCHO'] = trainProcesado['RANCHO_x'] | trainProcesado['RANCHO_y'] 
trainProcesado['PLAYA'] = trainProcesado['PLAYA_x'] | trainProcesado['PLAYA_y']
trainProcesado['GOLF'] = trainProcesado['GOLF_x'] | trainProcesado['GOLF_y']
#Seguridad
trainProcesado['SEGURIDAD'] = trainProcesado['SEGURIDAD_x'] | trainProcesado['SEGURIDAD_y'] 

#Borramos las columnas que quedaron del merge
trainProcesado.drop(list(trainProcesado.filter(regex = '_x')), axis = 1, inplace = True)
trainProcesado.drop(list(trainProcesado.filter(regex = '_y')), axis = 1, inplace = True)

In [48]:
trainProcesado.drop('idzona', axis = 1, inplace = True)
trainProcesado.drop('anio', axis = 1, inplace = True)

In [49]:
trainProcesado.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 240000 entries, 0 to 239999
Data columns (total 73 columns):
id                            240000 non-null int64
habitaciones                  240000 non-null int64
garages                       240000 non-null int64
banos                         240000 non-null int64
gimnasio                      240000 non-null int64
usosmultiples                 240000 non-null int64
piscina                       240000 non-null int64
escuelascercanas              240000 non-null int64
centroscomercialescercanos    240000 non-null int64
2012                          240000 non-null int64
2013                          240000 non-null int64
2014                          240000 non-null int64
2015                          240000 non-null int64
2016                          240000 non-null int64
precio                        240000 non-null float64
ciudad                        240000 non-null int64
provincia                     240000 non-null int64
tip

In [61]:
trainProcesado['mean'].fillna(trainProcesado['mean'].mean(), inplace = True)
trainProcesado['median'].fillna(trainProcesado['median'].mean(), inplace = True)

In [64]:
#Exportamos el DF a CSV, para usar como entrada de los distintos algoritmos probados
trainProcesado.to_csv('DATA/trainProcesado_patov5.csv', index=False)

# TEST

In [82]:
#Lectura de archivos CSV
test = pd.read_csv('../../DATA_TP2/test.csv')

In [83]:
#Nuevo dataframe
testProcesado = pd.DataFrame()

#Copiamos la columna con el id
testProcesado['id'] = test['id']
testProcesado['idzona'] = test['idzona']

#Manejo de NaN, columnas numericas
test['garages'].fillna(0, inplace=True)
test['banos'].fillna(1, inplace=True)
promedio_antiguedad_x_ciudad = test.groupby('ciudad')['antiguedad'].transform('mean')
test['antiguedad'].fillna(promedio_antiguedad_x_ciudad, inplace=True)
test['antiguedad'].fillna(test['antiguedad'].mean(), inplace=True)
promedio_habitaciones_x_ciudad = test.groupby('ciudad')['habitaciones'].transform('mean')
test['habitaciones'].fillna(promedio_habitaciones_x_ciudad, inplace=True)
test['habitaciones'].fillna(test['habitaciones'].mean(), inplace=True)
promedio_metroscubierto_x_ciudad = test.groupby('ciudad')['metroscubiertos'].transform('mean')
test['metroscubiertos'].fillna(promedio_metroscubierto_x_ciudad, inplace=True)
test['metroscubiertos'].fillna(test['metroscubiertos'].mean(), inplace=True)
promedio_metrostotales_x_ciudad = test.groupby('ciudad')['metrostotales'].transform('mean')
test['metrostotales'].fillna(promedio_metrostotales_x_ciudad, inplace=True)
test['metrostotales'].fillna(test['metrostotales'].mean(), inplace=True)

#Manejo de NaN, columnas catedoricas
test['ciudad'] = test['ciudad'].fillna('Desconocida')
test['provincia'] = test['provincia'].fillna('Desconocida')
test['tipodepropiedad'] = test['tipodepropiedad'].fillna('Desconocido')

test['ciudad'] = test['ciudad'].astype('category')
test['provincia'] = test['provincia'].astype('category')
test['tipodepropiedad'] = test['tipodepropiedad'].astype('category')

#Asignacion de tipos de datos
testProcesado['antiguedad'] = test['antiguedad'].astype('int64')
testProcesado['habitaciones'] = test['habitaciones'].astype('int64')
testProcesado['garages'] = test['garages'].astype('int64')
testProcesado['banos'] = test['banos'].astype('int64')
testProcesado['metroscubiertos'] = test['metroscubiertos'].astype('int64')
testProcesado['metrostotales'] = test['metrostotales'].astype('int64')

testProcesado['gimnasio'] = test['gimnasio'].astype('int64')
testProcesado['usosmultiples'] = test['usosmultiples'].astype('int64')
testProcesado['piscina'] = test['piscina'].astype('int64')
testProcesado['escuelascercanas'] = test['escuelascercanas'].astype('int64')
testProcesado['escuelascercanas'] = test['escuelascercanas'].astype('int64')
testProcesado['centroscomercialescercanos'] = test['centroscomercialescercanos'].astype('int64')

#Solo nos quedamos con el anio de la fecha
testProcesado['anio'] = pd.to_datetime(test['fecha'], infer_datetime_format=True).dt.year
testProcesado['2012'] = 0
testProcesado['2013'] = 0
testProcesado['2014'] = 0
testProcesado['2015'] = 0
testProcesado['2016'] = 0
testProcesado.loc[test['fecha'].str.contains('2012', na=False), '2012'] = 1
testProcesado.loc[test['fecha'].str.contains('2013', na=False), '2013'] = 1
testProcesado.loc[test['fecha'].str.contains('2014', na=False), '2014'] = 1
testProcesado.loc[test['fecha'].str.contains('2015', na=False), '2015'] = 1
testProcesado.loc[test['fecha'].str.contains('2016', na=False), '2016'] = 1

In [84]:
#Para las columas categorias
#Las guardamos con Label Encoding
cat_features = ['ciudad', 'provincia', 'tipodepropiedad']
encoder = LabelEncoder()
encoded = test[cat_features].apply(encoder.fit_transform)

testProcesado = testProcesado.join(encoded)
encoded

ciudad  provincia  tipodepropiedad
0         284          9                2
1         299         31                0
2         137          9                0
3           5         13                0
4         517         11                2
...       ...        ...              ...
59995     532          9                2
59996     162         11                2
59997     188         19                2
59998     352         22                2
59999     564         15                2

[60000 rows x 3 columns]

In [85]:
# AGRUPAMIENTO POR ANTIGUEDAD (BANDAS)
testProcesado['antiguedad_agrupada'] = 0

testProcesado.loc[testProcesado.antiguedad <= 5, 'antiguedad_agrupada'] = 0
testProcesado.loc[testProcesado.antiguedad > 95, 'antiguedad_agrupada'] = 100
testProcesado.loc[((testProcesado.antiguedad > 5) & (testProcesado.antiguedad <= 15)), 'antiguedad_agrupada'] = 10
testProcesado.loc[((testProcesado.antiguedad > 15) & (testProcesado.antiguedad <= 25)), 'antiguedad_agrupada'] = 20
testProcesado.loc[((testProcesado.antiguedad > 25) & (testProcesado.antiguedad <= 35)), 'antiguedad_agrupada'] = 30
testProcesado.loc[((testProcesado.antiguedad > 35) & (testProcesado.antiguedad <= 45)), 'antiguedad_agrupada'] = 40
testProcesado.loc[((testProcesado.antiguedad > 45) & (testProcesado.antiguedad <= 55)), 'antiguedad_agrupada'] = 50
testProcesado.loc[((testProcesado.antiguedad > 55) & (testProcesado.antiguedad <= 65)), 'antiguedad_agrupada'] = 60
testProcesado.loc[((testProcesado.antiguedad > 65) & (testProcesado.antiguedad <= 75)), 'antiguedad_agrupada'] = 70
testProcesado.loc[((testProcesado.antiguedad > 75) & (testProcesado.antiguedad <= 85)), 'antiguedad_agrupada'] = 80
testProcesado.loc[((testProcesado.antiguedad > 85) & (testProcesado.antiguedad <= 95)), 'antiguedad_agrupada'] = 90

testProcesado.drop('antiguedad', 1, inplace = True)

# AGRUPAMIENTO POR METROS CUBIENTOS (BANDAS)
testProcesado['metroscubiertos_agrupada'] = 0

testProcesado.loc[testProcesado.metroscubiertos <= 5, 'metroscubiertos_agrupada'] = 0
testProcesado.loc[testProcesado.metroscubiertos > 805, 'metroscubiertos_agrupada'] = 1000
testProcesado.loc[((testProcesado.metroscubiertos > 5) & (testProcesado.metroscubiertos <= 15)), 'metroscubiertos_agrupada'] = 10
testProcesado.loc[((testProcesado.metroscubiertos > 15) & (testProcesado.metroscubiertos <= 25)), 'metroscubiertos_agrupada'] = 20
testProcesado.loc[((testProcesado.metroscubiertos > 25) & (testProcesado.metroscubiertos <= 35)), 'metroscubiertos_agrupada'] = 30
testProcesado.loc[((testProcesado.metroscubiertos > 35) & (testProcesado.metroscubiertos <= 45)), 'metroscubiertos_agrupada'] = 40
testProcesado.loc[((testProcesado.metroscubiertos > 45) & (testProcesado.metroscubiertos <= 55)), 'metroscubiertos_agrupada'] = 50
testProcesado.loc[((testProcesado.metroscubiertos > 55) & (testProcesado.metroscubiertos <= 65)), 'metroscubiertos_agrupada'] = 60
testProcesado.loc[((testProcesado.metroscubiertos > 65) & (testProcesado.metroscubiertos <= 75)), 'metroscubiertos_agrupada'] = 70
testProcesado.loc[((testProcesado.metroscubiertos > 75) & (testProcesado.metroscubiertos <= 85)), 'metroscubiertos_agrupada'] = 80
testProcesado.loc[((testProcesado.metroscubiertos > 85) & (testProcesado.metroscubiertos <= 95)), 'metroscubiertos_agrupada'] = 90
testProcesado.loc[((testProcesado.metroscubiertos > 95) & (testProcesado.metroscubiertos <= 105)), 'metroscubiertos_agrupada'] = 100
testProcesado.loc[((testProcesado.metroscubiertos > 105) & (testProcesado.metroscubiertos <= 115)), 'metroscubiertos_agrupada'] = 110
testProcesado.loc[((testProcesado.metroscubiertos > 115) & (testProcesado.metroscubiertos <= 130)), 'metroscubiertos_agrupada'] = 125
testProcesado.loc[((testProcesado.metroscubiertos > 130) & (testProcesado.metroscubiertos <= 155)), 'metroscubiertos_agrupada'] = 150
testProcesado.loc[((testProcesado.metroscubiertos > 155) & (testProcesado.metroscubiertos <= 205)), 'metroscubiertos_agrupada'] = 200
testProcesado.loc[((testProcesado.metroscubiertos > 205) & (testProcesado.metroscubiertos <= 255)), 'metroscubiertos_agrupada'] = 250
testProcesado.loc[((testProcesado.metroscubiertos > 255) & (testProcesado.metroscubiertos <= 305)), 'metroscubiertos_agrupada'] = 300
testProcesado.loc[((testProcesado.metroscubiertos > 305) & (testProcesado.metroscubiertos <= 355)), 'metroscubiertos_agrupada'] = 350
testProcesado.loc[((testProcesado.metroscubiertos > 355) & (testProcesado.metroscubiertos <= 405)), 'metroscubiertos_agrupada'] = 400
testProcesado.loc[((testProcesado.metroscubiertos > 405) & (testProcesado.metroscubiertos <= 455)), 'metroscubiertos_agrupada'] = 450
testProcesado.loc[((testProcesado.metroscubiertos > 455) & (testProcesado.metroscubiertos <= 500)), 'metroscubiertos_agrupada'] = 500
testProcesado.loc[((testProcesado.metroscubiertos > 505) & (testProcesado.metroscubiertos <= 555)), 'metroscubiertos_agrupada'] = 550
testProcesado.loc[((testProcesado.metroscubiertos > 555) & (testProcesado.metroscubiertos <= 600)), 'metroscubiertos_agrupada'] = 600
testProcesado.loc[((testProcesado.metroscubiertos > 605) & (testProcesado.metroscubiertos <= 655)), 'metroscubiertos_agrupada'] = 650
testProcesado.loc[((testProcesado.metroscubiertos > 655) & (testProcesado.metroscubiertos <= 700)), 'metroscubiertos_agrupada'] = 700
testProcesado.loc[((testProcesado.metroscubiertos > 705) & (testProcesado.metroscubiertos <= 755)), 'metroscubiertos_agrupada'] = 750
testProcesado.loc[((testProcesado.metroscubiertos > 755) & (testProcesado.metroscubiertos <= 805)), 'metroscubiertos_agrupada'] = 800

testProcesado.drop('metroscubiertos', 1, inplace = True)

# AGRUPAMIENTO POR METROS TOTALES (BANDAS)
testProcesado['metrostotales_agrupada'] = 0

testProcesado.loc[testProcesado.metrostotales <= 5, 'metrostotales_agrupada'] = 0
testProcesado.loc[testProcesado.metrostotales > 805, 'metrostotales_agrupada'] = 1000
testProcesado.loc[((testProcesado.metrostotales > 5) & (testProcesado.metrostotales <= 15)), 'metrostotales_agrupada'] = 10
testProcesado.loc[((testProcesado.metrostotales > 15) & (testProcesado.metrostotales <= 25)), 'metrostotales_agrupada'] = 20
testProcesado.loc[((testProcesado.metrostotales > 25) & (testProcesado.metrostotales <= 35)), 'metrostotales_agrupada'] = 30
testProcesado.loc[((testProcesado.metrostotales > 35) & (testProcesado.metrostotales <= 45)), 'metrostotales_agrupada'] = 40
testProcesado.loc[((testProcesado.metrostotales > 45) & (testProcesado.metrostotales <= 55)), 'metrostotales_agrupada'] = 50
testProcesado.loc[((testProcesado.metrostotales > 55) & (testProcesado.metrostotales <= 65)), 'metrostotales_agrupada'] = 60
testProcesado.loc[((testProcesado.metrostotales > 65) & (testProcesado.metrostotales <= 75)), 'metrostotales_agrupada'] = 70
testProcesado.loc[((testProcesado.metrostotales > 75) & (testProcesado.metrostotales <= 85)), 'metrostotales_agrupada'] = 80
testProcesado.loc[((testProcesado.metrostotales > 85) & (testProcesado.metrostotales <= 95)), 'metrostotales_agrupada'] = 90
testProcesado.loc[((testProcesado.metrostotales > 95) & (testProcesado.metrostotales <= 105)), 'metrostotales_agrupada'] = 100
testProcesado.loc[((testProcesado.metrostotales > 105) & (testProcesado.metrostotales <= 115)), 'metrostotales_agrupada'] = 110
testProcesado.loc[((testProcesado.metrostotales > 115) & (testProcesado.metrostotales <= 130)), 'metrostotales_agrupada'] = 125
testProcesado.loc[((testProcesado.metrostotales > 130) & (testProcesado.metrostotales <= 155)), 'metrostotales_agrupada'] = 150
testProcesado.loc[((testProcesado.metrostotales > 155) & (testProcesado.metrostotales <= 205)), 'metrostotales_agrupada'] = 200
testProcesado.loc[((testProcesado.metrostotales > 205) & (testProcesado.metrostotales <= 255)), 'metrostotales_agrupada'] = 250
testProcesado.loc[((testProcesado.metrostotales > 255) & (testProcesado.metrostotales <= 305)), 'metrostotales_agrupada'] = 300
testProcesado.loc[((testProcesado.metrostotales > 305) & (testProcesado.metrostotales <= 355)), 'metrostotales_agrupada'] = 350
testProcesado.loc[((testProcesado.metrostotales > 355) & (testProcesado.metrostotales <= 405)), 'metrostotales_agrupada'] = 400
testProcesado.loc[((testProcesado.metrostotales > 405) & (testProcesado.metrostotales <= 455)), 'metrostotales_agrupada'] = 450
testProcesado.loc[((testProcesado.metrostotales > 455) & (testProcesado.metrostotales <= 500)), 'metrostotales_agrupada'] = 500
testProcesado.loc[((testProcesado.metrostotales > 505) & (testProcesado.metrostotales <= 555)), 'metrostotales_agrupada'] = 550
testProcesado.loc[((testProcesado.metrostotales > 555) & (testProcesado.metrostotales <= 600)), 'metrostotales_agrupada'] = 600
testProcesado.loc[((testProcesado.metrostotales > 605) & (testProcesado.metrostotales <= 655)), 'metrostotales_agrupada'] = 650
testProcesado.loc[((testProcesado.metrostotales > 655) & (testProcesado.metrostotales <= 700)), 'metrostotales_agrupada'] = 700
testProcesado.loc[((testProcesado.metrostotales > 705) & (testProcesado.metrostotales <= 755)), 'metrostotales_agrupada'] = 750
testProcesado.loc[((testProcesado.metrostotales > 755) & (testProcesado.metrostotales <= 805)), 'metrostotales_agrupada'] = 800

testProcesado.drop('metrostotales', 1, inplace = True)


In [86]:
testProcesado = pd.merge(testProcesado, train_gruouped, how='left', left_on=['idzona','tipodepropiedad','habitaciones','anio'], right_on = ['idzona','tipodepropiedad','habitaciones','anio'])

In [87]:
#Procesamiento de columna titulo
#Parseamos las palabras seleccionadas en el DF (ver notebook de exploracion)

#Nuevo dataframe
titleFeatures = pd.DataFrame()
titleFeatures['id'] = test['id']
#Uppercase
titleFeatures['titulo'] = test['titulo'].str.upper()
#Limpiamos las columnas
titleFeatures['titulo'] = titleFeatures['titulo'].str.normalize('NFKD')\
                                                .str.encode('ascii', errors='ignore')\
                                                .str.decode('utf-8')

#Armamos las columnas para cada palabra seleccionada
#Valor de inicializacion

#Propiedad nueva
titleFeatures['NUEVA'] = 0
#Calificativo (no aporta mucho)
titleFeatures['EXCELENTE'] = 0
#Tipo de aviso
titleFeatures['VENTA'] = 0
titleFeatures['PREVENTA'] = 0
titleFeatures['RENTA'] = 0
titleFeatures['CREDITO'] = 0
titleFeatures['REMATE'] = 0
titleFeatures['OPORTUNIDAD'] = 0
#Tipo de propiedad -extra
titleFeatures['RESIDENCIA'] = 0
titleFeatures['EDIFICIO'] = 0
titleFeatures['TORRE'] = 0
titleFeatures['PISO'] = 0
titleFeatures['LOFT'] = 0
titleFeatures['FRACCIONAMIENTO'] = 0
titleFeatures['DUPLEX'] = 0
titleFeatures['COUNTRY'] = 0
titleFeatures['QUINTA'] = 0
titleFeatures['OFICINA'] = 0
titleFeatures['PENTHOUSE'] = 0
#Tipo de propiedad -extra 2
titleFeatures['REMODELADO'] = 0
titleFeatures['MODERNO'] = 0
titleFeatures['MINIMALISTA'] = 0
titleFeatures['EQUIPADA'] = 0
#Jardin
titleFeatures['JARDIN'] = 0
#Ubicacion
titleFeatures['FRENTE'] = 0
titleFeatures['ESQUINA'] = 0
titleFeatures['CENTRO'] = 0
titleFeatures['UBICACION'] = 0
#Vistas o panoramicas
titleFeatures['VISTA'] = 0
#Caracteristicas extra
titleFeatures['LUJO'] = 0
titleFeatures['BODEGA'] = 0
titleFeatures['CAMPESTRE'] = 0
titleFeatures['RANCHO'] = 0
titleFeatures['PLAYA'] = 0
titleFeatures['GOLF'] = 0
#Seguridad
titleFeatures['SEGURIDAD'] = 0

#Buscamos y agregamos 1 donde aparece la palabra
titleFeatures.loc[titleFeatures['titulo'].str.contains('NUEVA', na=False), 'NUEVA'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('NUEVO', na=False), 'NUEVA'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('NUEVAS', na=False), 'NUEVA'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('ESTRENE', na=False), 'NUEVA'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('ESTRENAR', na=False), 'NUEVA'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('NUEVOS', na=False), 'NUEVA'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('ESTRENA', na=False), 'NUEVA'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('HERMOSA', na=False), 'EXCELENTE'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('EXELENTE', na=False), 'EXCELENTE'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('EXCELENTE', na=False), 'EXCELENTE'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('PRECIOSA', na=False), 'EXCELENTE'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('HERMOSO', na=False), 'EXCELENTE'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('BONITO', na=False), 'EXCELENTE'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('EXCELENTES', na=False), 'EXCELENTE'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('INMEJORABLE', na=False), 'EXCELENTE'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('AGRADABLE', na=False), 'EXCELENTE'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('LINDA', na=False), 'EXCELENTE'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('PRECIOSO', na=False), 'EXCELENTE'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('MAGNIFICA', na=False), 'EXCELENTE'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('IMPECABLE', na=False), 'EXCELENTE'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('HERMOSAS', na=False), 'EXCELENTE'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('EXCLUSIVA', na=False), 'EXCELENTE'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('ESPECTACULAR', na=False), 'EXCELENTE'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('INCREIBLE', na=False), 'EXCELENTE'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('EXCLUSIVO', na=False), 'EXCELENTE'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('SUPER', na=False), 'EXCELENTE'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('LINDO', na=False), 'EXCELENTE'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('MAGNIFICO', na=False), 'EXCELENTE'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('VENTA', na=False), 'VENTA'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('PREVENTA', na=False), 'PREVENTA'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('PRE-VENTA', na=False), 'PREVENTA'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('RENTA', na=False), 'RENTA'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('CREDITO', na=False), 'CREDITO'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('HIPOTECARIO', na=False), 'CREDITO'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('REMATE', na=False), 'REMATE'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('REMATO', na=False), 'REMATE'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('OPORTUNIDAD', na=False), 'OPORTUNIDAD'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('RESIDENCIA', na=False), 'RESIDENCIA'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('RESIDENCIAL', na=False), 'RESIDENCIA'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('EDIFICIO', na=False), 'EDIFICIO'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('TORRE', na=False), 'TORRE'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('PISO', na=False), 'PISO'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('LOFT', na=False), 'LOFT'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('FRACCIONAMIENTO', na=False), 'FRACCIONAMIENTO'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('FRACC', na=False), 'FRACCIONAMIENTO'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('DUPLEX', na=False), 'DUPLEX'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('COUNTRY', na=False), 'COUNTRY'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('QUINTA', na=False), 'QUINTA'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('OFICINA', na=False), 'OFICINA'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('PENTHOUSE', na=False), 'PENTHOUSE'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('REMODELADO', na=False), 'REMODELADO'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('REMODELADA', na=False), 'REMODELADO'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('MODERNA', na=False), 'MODERNO'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('MODERNO', na=False), 'MODERNO'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('MINIMALISTA', na=False), 'MINIMALISTA'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('EQUIPADA', na=False), 'EQUIPADA'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('JARDINES', na=False), 'JARDIN'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('JARDIN', na=False), 'JARDIN'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('VERDES', na=False), 'JARDIN'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('ARBOLEDAS', na=False), 'JARDIN'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('PARQUES', na=False), 'JARDIN'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('PATIO', na=False), 'JARDIN'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('FRENTE', na=False), 'FRENTE'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('ESQUINA', na=False), 'ESQUINA'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('CENTRO', na=False), 'CENTRO'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('CENTRAL', na=False), 'CENTRO'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('UBICACION', na=False), 'UBICACION'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('UBICADISIMO', na=False), 'UBICACION'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('PANORAMICA', na=False), 'VISTA'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('VISTA', na=False), 'VISTA'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('VISTAS', na=False), 'VISTA'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('LUJO', na=False), 'LUJO'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('BODEGA', na=False), 'BODEGA'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('CAMPESTRE', na=False), 'CAMPESTRE'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('RANCHO', na=False), 'RANCHO'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('PLAYA', na=False), 'PLAYA'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('GOLF', na=False), 'GOLF'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('SEGURIDAD', na=False), 'SEGURIDAD'] = 1
titleFeatures.loc[titleFeatures['titulo'].str.contains('VIGILANCIA', na=False), 'SEGURIDAD'] = 1

#Alternativamente, llenamos una sola columna como categorica, para luego encodear con LE
titleFeatures['titulo_features'] = 'NINGUNA'
titleFeatures.loc[titleFeatures['titulo'].str.contains('NUEVA', na=False), 'titulo_features'] = 'NUEVA'
titleFeatures.loc[titleFeatures['titulo'].str.contains('NUEVO', na=False), 'titulo_features'] = 'NUEVA'
titleFeatures.loc[titleFeatures['titulo'].str.contains('NUEVAS', na=False), 'titulo_features'] = 'NUEVA'
titleFeatures.loc[titleFeatures['titulo'].str.contains('ESTRENE', na=False), 'titulo_features'] = 'NUEVA'
titleFeatures.loc[titleFeatures['titulo'].str.contains('ESTRENAR', na=False), 'titulo_features'] = 'NUEVA'
titleFeatures.loc[titleFeatures['titulo'].str.contains('NUEVOS', na=False), 'titulo_features'] = 'NUEVA'
titleFeatures.loc[titleFeatures['titulo'].str.contains('ESTRENA', na=False), 'titulo_features'] = 'NUEVA'
titleFeatures.loc[titleFeatures['titulo'].str.contains('HERMOSA', na=False), 'titulo_features'] = 'EXCELENTE'
titleFeatures.loc[titleFeatures['titulo'].str.contains('EXELENTE', na=False), 'titulo_features'] = 'EXCELENTE'
titleFeatures.loc[titleFeatures['titulo'].str.contains('EXCELENTE', na=False), 'titulo_features'] = 'EXCELENTE'
titleFeatures.loc[titleFeatures['titulo'].str.contains('PRECIOSA', na=False), 'titulo_features'] = 'EXCELENTE'
titleFeatures.loc[titleFeatures['titulo'].str.contains('HERMOSO', na=False), 'titulo_features'] = 'EXCELENTE'
titleFeatures.loc[titleFeatures['titulo'].str.contains('BONITO', na=False), 'titulo_features'] = 'EXCELENTE'
titleFeatures.loc[titleFeatures['titulo'].str.contains('EXCELENTES', na=False), 'titulo_features'] = 'EXCELENTE'
titleFeatures.loc[titleFeatures['titulo'].str.contains('INMEJORABLE', na=False), 'titulo_features'] = 'EXCELENTE'
titleFeatures.loc[titleFeatures['titulo'].str.contains('AGRADABLE', na=False), 'titulo_features'] = 'EXCELENTE'
titleFeatures.loc[titleFeatures['titulo'].str.contains('LINDA', na=False), 'titulo_features'] = 'EXCELENTE'
titleFeatures.loc[titleFeatures['titulo'].str.contains('PRECIOSO', na=False), 'titulo_features'] = 'EXCELENTE'
titleFeatures.loc[titleFeatures['titulo'].str.contains('MAGNIFICA', na=False), 'titulo_features'] = 'EXCELENTE'
titleFeatures.loc[titleFeatures['titulo'].str.contains('IMPECABLE', na=False), 'titulo_features'] = 'EXCELENTE'
titleFeatures.loc[titleFeatures['titulo'].str.contains('HERMOSAS', na=False), 'titulo_features'] = 'EXCELENTE'
titleFeatures.loc[titleFeatures['titulo'].str.contains('EXCLUSIVA', na=False), 'titulo_features'] = 'EXCELENTE'
titleFeatures.loc[titleFeatures['titulo'].str.contains('ESPECTACULAR', na=False), 'titulo_features'] = 'EXCELENTE'
titleFeatures.loc[titleFeatures['titulo'].str.contains('INCREIBLE', na=False), 'titulo_features'] = 'EXCELENTE'
titleFeatures.loc[titleFeatures['titulo'].str.contains('EXCLUSIVO', na=False), 'titulo_features'] = 'EXCELENTE'
titleFeatures.loc[titleFeatures['titulo'].str.contains('SUPER', na=False), 'titulo_features'] = 'EXCELENTE'
titleFeatures.loc[titleFeatures['titulo'].str.contains('LINDO', na=False), 'titulo_features'] = 'EXCELENTE'
titleFeatures.loc[titleFeatures['titulo'].str.contains('MAGNIFICO', na=False), 'titulo_features'] = 'EXCELENTE'
titleFeatures.loc[titleFeatures['titulo'].str.contains('VENTA', na=False), 'titulo_features'] = 'VENTA'
titleFeatures.loc[titleFeatures['titulo'].str.contains('PREVENTA', na=False), 'titulo_features'] = 'PREVENTA'
titleFeatures.loc[titleFeatures['titulo'].str.contains('PRE-VENTA', na=False), 'titulo_features'] = 'PREVENTA'
titleFeatures.loc[titleFeatures['titulo'].str.contains('RENTA', na=False), 'titulo_features'] = 'RENTA'
titleFeatures.loc[titleFeatures['titulo'].str.contains('CREDITO', na=False), 'titulo_features'] = 'CREDITO'
titleFeatures.loc[titleFeatures['titulo'].str.contains('HIPOTECARIO', na=False), 'titulo_features'] = 'CREDITO'
titleFeatures.loc[titleFeatures['titulo'].str.contains('REMATE', na=False), 'titulo_features'] = 'REMATE'
titleFeatures.loc[titleFeatures['titulo'].str.contains('REMATO', na=False), 'titulo_features'] = 'REMATE'
titleFeatures.loc[titleFeatures['titulo'].str.contains('OPORTUNIDAD', na=False), 'titulo_features'] = 'OPORTUNIDAD'
titleFeatures.loc[titleFeatures['titulo'].str.contains('RESIDENCIA', na=False), 'titulo_features'] = 'RESIDENCIA'
titleFeatures.loc[titleFeatures['titulo'].str.contains('RESIDENCIAL', na=False), 'titulo_features'] = 'RESIDENCIA'
titleFeatures.loc[titleFeatures['titulo'].str.contains('EDIFICIO', na=False), 'titulo_features'] = 'EDIFICIO'
titleFeatures.loc[titleFeatures['titulo'].str.contains('TORRE', na=False), 'titulo_features'] = 'TORRE'
titleFeatures.loc[titleFeatures['titulo'].str.contains('PISO', na=False), 'titulo_features'] = 'PISO'
titleFeatures.loc[titleFeatures['titulo'].str.contains('LOFT', na=False), 'titulo_features'] = 'LOFT'
titleFeatures.loc[titleFeatures['titulo'].str.contains('FRACCIONAMIENTO', na=False), 'titulo_features'] = 'FRACCIONAMIENTO'
titleFeatures.loc[titleFeatures['titulo'].str.contains('FRACC', na=False), 'titulo_features'] = 'FRACCIONAMIENTO'
titleFeatures.loc[titleFeatures['titulo'].str.contains('DUPLEX', na=False), 'titulo_features'] = 'DUPLEX'
titleFeatures.loc[titleFeatures['titulo'].str.contains('COUNTRY', na=False), 'titulo_features'] = 'COUNTRY'
titleFeatures.loc[titleFeatures['titulo'].str.contains('QUINTA', na=False), 'titulo_features'] = 'QUINTA'
titleFeatures.loc[titleFeatures['titulo'].str.contains('OFICINA', na=False), 'titulo_features'] = 'OFICINA'
titleFeatures.loc[titleFeatures['titulo'].str.contains('PENTHOUSE', na=False), 'titulo_features'] = 'PENTHOUSE'
titleFeatures.loc[titleFeatures['titulo'].str.contains('REMODELADO', na=False), 'titulo_features'] = 'REMODELADO'
titleFeatures.loc[titleFeatures['titulo'].str.contains('REMODELADA', na=False), 'titulo_features'] = 'REMODELADO'
titleFeatures.loc[titleFeatures['titulo'].str.contains('MODERNA', na=False), 'titulo_features'] = 'MODERNO'
titleFeatures.loc[titleFeatures['titulo'].str.contains('MODERNO', na=False), 'titulo_features'] = 'MODERNO'
titleFeatures.loc[titleFeatures['titulo'].str.contains('MINIMALISTA', na=False), 'titulo_features'] = 'MINIMALISTA'
titleFeatures.loc[titleFeatures['titulo'].str.contains('EQUIPADA', na=False), 'titulo_features'] = 'EQUIPADA'
titleFeatures.loc[titleFeatures['titulo'].str.contains('JARDINES', na=False), 'titulo_features'] = 'JARDIN'
titleFeatures.loc[titleFeatures['titulo'].str.contains('JARDIN', na=False), 'titulo_features'] = 'JARDIN'
titleFeatures.loc[titleFeatures['titulo'].str.contains('VERDES', na=False), 'titulo_features'] = 'JARDIN'
titleFeatures.loc[titleFeatures['titulo'].str.contains('ARBOLEDAS', na=False), 'titulo_features'] = 'JARDIN'
titleFeatures.loc[titleFeatures['titulo'].str.contains('PARQUES', na=False), 'titulo_features'] = 'JARDIN'
titleFeatures.loc[titleFeatures['titulo'].str.contains('PATIO', na=False), 'titulo_features'] = 'JARDIN'
titleFeatures.loc[titleFeatures['titulo'].str.contains('FRENTE', na=False), 'titulo_features'] = 'FRENTE'
titleFeatures.loc[titleFeatures['titulo'].str.contains('ESQUINA', na=False), 'titulo_features'] = 'ESQUINA'
titleFeatures.loc[titleFeatures['titulo'].str.contains('CENTRO', na=False), 'titulo_features'] = 'CENTRO'
titleFeatures.loc[titleFeatures['titulo'].str.contains('CENTRAL', na=False), 'titulo_features'] = 'CENTRO'
titleFeatures.loc[titleFeatures['titulo'].str.contains('UBICACION', na=False), 'titulo_features'] = 'UBICACION'
titleFeatures.loc[titleFeatures['titulo'].str.contains('UBICADISIMO', na=False), 'titulo_features'] = 'UBICACION'
titleFeatures.loc[titleFeatures['titulo'].str.contains('PANORAMICA', na=False), 'titulo_features'] = 'VISTA'
titleFeatures.loc[titleFeatures['titulo'].str.contains('VISTA', na=False), 'titulo_features'] = 'VISTA'
titleFeatures.loc[titleFeatures['titulo'].str.contains('VISTAS', na=False), 'titulo_features'] = 'VISTA'
titleFeatures.loc[titleFeatures['titulo'].str.contains('LUJO', na=False), 'titulo_features'] = 'LUJO'
titleFeatures.loc[titleFeatures['titulo'].str.contains('BODEGA', na=False), 'titulo_features'] = 'BODEGA'
titleFeatures.loc[titleFeatures['titulo'].str.contains('CAMPESTRE', na=False), 'titulo_features'] = 'CAMPESTRE'
titleFeatures.loc[titleFeatures['titulo'].str.contains('RANCHO', na=False), 'titulo_features'] = 'RANCHO'
titleFeatures.loc[titleFeatures['titulo'].str.contains('PLAYA', na=False), 'titulo_features'] = 'PLAYA'
titleFeatures.loc[titleFeatures['titulo'].str.contains('GOLF', na=False), 'titulo_features'] = 'GOLF'
titleFeatures.loc[titleFeatures['titulo'].str.contains('SEGURIDAD', na=False), 'titulo_features'] = 'SEGURIDAD'
titleFeatures.loc[titleFeatures['titulo'].str.contains('VIGILANCIA', na=False), 'titulo_features'] = 'SEGURIDAD'
#La tipamos
titleFeatures['titulo_features'] = titleFeatures['titulo_features'].astype('category')

#La guardamos con label encoding
cat_features_titulo = ['titulo_features']
encoder_titulo = LabelEncoder()
encoded_titulo = titleFeatures[cat_features_titulo].apply(encoder_titulo.fit_transform)
encoded_titulo['id'] = test['id']
encoded_titulo = encoded_titulo.rename(columns={'titulo_features': 'titulo_features_enc'})

#Agregamos la columna al los DF
titleFeatures = pd.merge(titleFeatures,encoded_titulo, on='id', how='inner')
testProcesado = pd.merge(testProcesado,titleFeatures, on='id', how='inner')
testProcesado = testProcesado.drop('titulo',1)
testProcesado = testProcesado.drop('titulo_features',1)
testProcesado = testProcesado.drop('titulo_features_enc',1)

In [88]:
#Tratamiento analogo para la columna descripcion

#Procesamiento de columna descripcion
#Parseamos las palabras seleccionadas en el DF (ver notebook de exploracion)

#Nuevo dataframe
descFeatures = pd.DataFrame()
descFeatures['id'] = test['id']
#Uppercase
descFeatures['descripcion'] = test['descripcion'].str.upper()
#Limpiamos las columnas
descFeatures['descripcion'] = descFeatures['descripcion'].str.normalize('NFKD')\
                                                .str.encode('ascii', errors='ignore')\
                                                .str.decode('utf-8')

#Armamos las columnas para cada palabra seleccionada
#Valor de inicializacion

#Ya usadas en titulo
#Propiedad nueva
descFeatures['NUEVA'] = 0
#Calificativo (no aporta mucho)
descFeatures['EXCELENTE'] = 0
#Tipo de aviso
descFeatures['VENTA'] = 0
descFeatures['PREVENTA'] = 0
descFeatures['RENTA'] = 0
descFeatures['CREDITO'] = 0
descFeatures['REMATE'] = 0
descFeatures['OPORTUNIDAD'] = 0
#Tipo de propiedad -extra
descFeatures['RESIDENCIA'] = 0
descFeatures['EDIFICIO'] = 0
descFeatures['TORRE'] = 0
descFeatures['PISO'] = 0
descFeatures['LOFT'] = 0
descFeatures['FRACCIONAMIENTO'] = 0
descFeatures['DUPLEX'] = 0
descFeatures['COUNTRY'] = 0
descFeatures['QUINTA'] = 0
descFeatures['OFICINA'] = 0
descFeatures['PENTHOUSE'] = 0
#Tipo de propiedad -extra 2
descFeatures['REMODELADO'] = 0
descFeatures['MODERNO'] = 0
descFeatures['MINIMALISTA'] = 0
descFeatures['EQUIPADA'] = 0
#Jardin
descFeatures['JARDIN'] = 0
#Ubicacion
descFeatures['FRENTE'] = 0
descFeatures['ESQUINA'] = 0
descFeatures['CENTRO'] = 0
descFeatures['UBICACION'] = 0
#Vistas o panoramicas
descFeatures['VISTA'] = 0
#Caracteristicas extra
descFeatures['LUJO'] = 0
descFeatures['BODEGA'] = 0
descFeatures['CAMPESTRE'] = 0
descFeatures['RANCHO'] = 0
descFeatures['PLAYA'] = 0
descFeatures['GOLF'] = 0
#Seguridad
descFeatures['SEGURIDAD'] = 0

#Nuevas de descripcion
#Iluminacion
descFeatures['ILUMINADA'] = 0
#Ubicacion
descFeatures['TRANQUILO'] = 0
descFeatures['UNIVERSIDAD'] = 0
descFeatures['AVENIDA'] = 0
#Caracteristicas extra
descFeatures['AMENIDADES'] = 0
descFeatures['BARRA'] = 0
descFeatures['BALCON'] = 0
descFeatures['RECIBIDOR'] = 0
descFeatures['AMENIDADES'] = 0
descFeatures['DESPENSA'] = 0
descFeatures['LAVADERO'] = 0
descFeatures['REJA'] = 0
descFeatures['TERRAZA'] = 0
descFeatures['CHIMENEA'] = 0
descFeatures['CISTERNA'] = 0

#Buscamos y agregamos 1 donde aparece la palabra
descFeatures.loc[descFeatures['descripcion'].str.contains('NUEVO', na=False), 'NUEVA'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('ESTRENAR', na=False), 'NUEVA'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('EXCELENTE', na=False), 'EXCELENTE'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('HERMOSA', na=False), 'EXCELENTE'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('BONITA', na=False), 'EXCELENTE'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('PRECIOSA', na=False), 'EXCELENTE'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('HERMOSO', na=False), 'EXCELENTE'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('AGRADABLE', na=False), 'EXCELENTE'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('PRECIOSO', na=False), 'EXCELENTE'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('ESPECTACULAR', na=False), 'EXCELENTE'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('HERMOSAS', na=False), 'EXCELENTE'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('HERMOSOS', na=False), 'EXCELENTE'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('BONITOS', na=False), 'EXCELENTE'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('INMEJORABLE', na=False), 'EXCELENTE'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('REMATES', na=False), 'REMATE'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('HIPOTECARIO', na=False), 'CREDITO'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('INVERSION', na=False), 'OPORTUNIDAD'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('INVERSIONISTAS', na=False), 'OPORTUNIDAD'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('RESIDENCIAL', na=False), 'RESIDENCIA'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('FRACCIONAMIENTO', na=False), 'FRACCIONAMIENTO'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('ESTANCIA', na=False), 'RANCHO'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('PISO', na=False), 'PISO'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('TORRE', na=False), 'TORRE'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('LUZ', na=False), 'ILUMINADA'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('ILUMINADA', na=False), 'ILUMINADA'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('ILUMINADO', na=False), 'ILUMINADA'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('ILUMINACION', na=False), 'ILUMINADA'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('PATIO', na=False), 'JARDIN'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('FONDO', na=False), 'JARDIN'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('JARDIN', na=False), 'JARDIN'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('VERDES', na=False), 'JARDIN'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('VERDE', na=False), 'JARDIN'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('GARDEN', na=False), 'JARDIN'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('PARQUE', na=False), 'JARDIN'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('NATURAL', na=False), 'JARDIN'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('BOSQUES', na=False), 'JARDIN'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('PARQUES', na=False), 'JARDIN'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('ARBOLES', na=False), 'JARDIN'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('VIGILANCIA', na=False), 'SEGURIDAD'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('SEGURIDAD', na=False), 'SEGURIDAD'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('SEGURO', na=False), 'SEGURIDAD'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('VISTAS', na=False), 'VISTA'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('PANORAMICA', na=False), 'VISTA'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('VISTA', na=False), 'VISTA'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('UBICACION', na=False), 'UBICACION'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('UBICADO', na=False), 'UBICACION'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('TRANQUILA', na=False), 'TRANQUILO'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('TRANQUILO', na=False), 'TRANQUILO'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('TRANQUILIDAD', na=False), 'TRANQUILO'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('UNIVERSIDAD', na=False), 'UNIVERSIDAD'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('AVENIDA', na=False), 'AVENIDA'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('ESQUINA', na=False), 'ESQUINA'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('FACHADA', na=False), 'FRENTE'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('FRENTE', na=False), 'FRENTE'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('REMODELADO', na=False), 'REMODELADO'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('REMODELADA', na=False), 'REMODELADO'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('MODERNA', na=False), 'MODERNO'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('EXCLUSIVA', na=False), 'LUJO'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('LUJO', na=False), 'LUJO'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('CALIDAD', na=False), 'LUJO'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('MARMOL', na=False), 'LUJO'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('PORCELANATO', na=False), 'LUJO'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('JACUZZI', na=False), 'LUJO'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('VESTIDOR', na=False), 'LUJO'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('GOLF', na=False), 'GOLF'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('BODEGA', na=False), 'BODEGA'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('PLAYA', na=False), 'PLAYA'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('AMENIDADES ', na=False), 'AMENIDADES'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('FIESTAS', na=False), 'AMENIDADES'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('EVENTOS', na=False), 'AMENIDADES'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('LAVANDERIA', na=False), 'AMENIDADES'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('JUEGOS', na=False), 'AMENIDADES'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('SERVICIOS', na=False), 'AMENIDADES'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('SERVICIO', na=False), 'AMENIDADES'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('ASADOR', na=False), 'AMENIDADES'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('CANCHAS', na=False), 'AMENIDADES'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('TENIS', na=False), 'AMENIDADES'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('BARRA', na=False), 'BARRA'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('DESAYUNADORA', na=False), 'BARRA'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('DESAYUNADOR', na=False), 'BARRA'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('BALCON', na=False), 'BALCON'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('RECIBIDOR', na=False), 'RECIBIDOR'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('VESTIBULO', na=False), 'RECIBIDOR'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('CHIMENEA', na=False), 'CHIMENEA'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('DESPENSA', na=False), 'DESPENSA'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('LAVADERO', na=False), 'LAVADERO'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('REJA', na=False), 'REJA'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('BOILER', na=False), 'CISTERNA'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('CISTERNA', na=False), 'CISTERNA'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('AZOTEA', na=False), 'TERRAZA'] = 1
descFeatures.loc[descFeatures['descripcion'].str.contains('TERRAZA', na=False), 'TERRAZA'] = 1

#Alternativamente, llenamos una sola columna como categorica, para luego encodear con LE
descFeatures['descripcion_features'] = 'NINGUNA'
descFeatures.loc[descFeatures['descripcion'].str.contains('NUEVO', na=False), 'descripcion_features'] = 'NUEVA'
descFeatures.loc[descFeatures['descripcion'].str.contains('ESTRENAR', na=False), 'descripcion_features'] = 'NUEVA'
descFeatures.loc[descFeatures['descripcion'].str.contains('EXCELENTE', na=False), 'descripcion_features'] = 'EXCELENTE'
descFeatures.loc[descFeatures['descripcion'].str.contains('HERMOSA', na=False), 'descripcion_features'] = 'EXCELENTE'
descFeatures.loc[descFeatures['descripcion'].str.contains('BONITA', na=False), 'descripcion_features'] = 'EXCELENTE'
descFeatures.loc[descFeatures['descripcion'].str.contains('PRECIOSA', na=False), 'descripcion_features'] = 'EXCELENTE'
descFeatures.loc[descFeatures['descripcion'].str.contains('HERMOSO', na=False), 'descripcion_features'] = 'EXCELENTE'
descFeatures.loc[descFeatures['descripcion'].str.contains('AGRADABLE', na=False), 'descripcion_features'] = 'EXCELENTE'
descFeatures.loc[descFeatures['descripcion'].str.contains('PRECIOSO', na=False), 'descripcion_features'] = 'EXCELENTE'
descFeatures.loc[descFeatures['descripcion'].str.contains('ESPECTACULAR', na=False), 'descripcion_features'] = 'EXCELENTE'
descFeatures.loc[descFeatures['descripcion'].str.contains('HERMOSAS', na=False), 'descripcion_features'] = 'EXCELENTE'
descFeatures.loc[descFeatures['descripcion'].str.contains('HERMOSOS', na=False), 'descripcion_features'] = 'EXCELENTE'
descFeatures.loc[descFeatures['descripcion'].str.contains('BONITOS', na=False), 'descripcion_features'] = 'EXCELENTE'
descFeatures.loc[descFeatures['descripcion'].str.contains('INMEJORABLE', na=False), 'descripcion_features'] = 'EXCELENTE'
descFeatures.loc[descFeatures['descripcion'].str.contains('REMATES', na=False), 'descripcion_features'] = 'REMATE'
descFeatures.loc[descFeatures['descripcion'].str.contains('HIPOTECARIO', na=False), 'descripcion_features'] = 'CREDITO'
descFeatures.loc[descFeatures['descripcion'].str.contains('INVERSION', na=False), 'descripcion_features'] = 'OPORTUNIDAD'
descFeatures.loc[descFeatures['descripcion'].str.contains('INVERSIONISTAS', na=False), 'descripcion_features'] = 'OPORTUNIDAD'
descFeatures.loc[descFeatures['descripcion'].str.contains('RESIDENCIAL', na=False), 'descripcion_features'] = 'RESIDENCIA'
descFeatures.loc[descFeatures['descripcion'].str.contains('FRACCIONAMIENTO', na=False), 'descripcion_features'] = 'FRACCIONAMIENTO'
descFeatures.loc[descFeatures['descripcion'].str.contains('ESTANCIA', na=False), 'descripcion_features'] = 'RANCHO'
descFeatures.loc[descFeatures['descripcion'].str.contains('PISO', na=False), 'descripcion_features'] = 'PISO'
descFeatures.loc[descFeatures['descripcion'].str.contains('TORRE', na=False), 'descripcion_features'] = 'TORRE'
descFeatures.loc[descFeatures['descripcion'].str.contains('LUZ', na=False), 'descripcion_features'] = 'ILUMINADA'
descFeatures.loc[descFeatures['descripcion'].str.contains('ILUMINADA', na=False), 'descripcion_features'] = 'ILUMINADA'
descFeatures.loc[descFeatures['descripcion'].str.contains('ILUMINADO', na=False), 'descripcion_features'] = 'ILUMINADA'
descFeatures.loc[descFeatures['descripcion'].str.contains('ILUMINACION', na=False), 'descripcion_features'] = 'ILUMINADA'
descFeatures.loc[descFeatures['descripcion'].str.contains('PATIO', na=False), 'descripcion_features'] = 'JARDIN'
descFeatures.loc[descFeatures['descripcion'].str.contains('FONDO', na=False), 'descripcion_features'] = 'JARDIN'
descFeatures.loc[descFeatures['descripcion'].str.contains('JARDIN', na=False), 'descripcion_features'] = 'JARDIN'
descFeatures.loc[descFeatures['descripcion'].str.contains('VERDES', na=False), 'descripcion_features'] = 'JARDIN'
descFeatures.loc[descFeatures['descripcion'].str.contains('VERDE', na=False), 'descripcion_features'] = 'JARDIN'
descFeatures.loc[descFeatures['descripcion'].str.contains('GARDEN', na=False), 'descripcion_features'] = 'JARDIN'
descFeatures.loc[descFeatures['descripcion'].str.contains('PARQUE', na=False), 'descripcion_features'] = 'JARDIN'
descFeatures.loc[descFeatures['descripcion'].str.contains('NATURAL', na=False), 'descripcion_features'] = 'JARDIN'
descFeatures.loc[descFeatures['descripcion'].str.contains('BOSQUES', na=False), 'descripcion_features'] = 'JARDIN'
descFeatures.loc[descFeatures['descripcion'].str.contains('PARQUES', na=False), 'descripcion_features'] = 'JARDIN'
descFeatures.loc[descFeatures['descripcion'].str.contains('ARBOLES', na=False), 'descripcion_features'] = 'JARDIN'
descFeatures.loc[descFeatures['descripcion'].str.contains('VIGILANCIA', na=False), 'descripcion_features'] = 'SEGURIDAD'
descFeatures.loc[descFeatures['descripcion'].str.contains('SEGURIDAD', na=False), 'descripcion_features'] = 'SEGURIDAD'
descFeatures.loc[descFeatures['descripcion'].str.contains('SEGURO', na=False), 'descripcion_features'] = 'SEGURIDAD'
descFeatures.loc[descFeatures['descripcion'].str.contains('VISTAS', na=False), 'descripcion_features'] = 'VISTA'
descFeatures.loc[descFeatures['descripcion'].str.contains('PANORAMICA', na=False), 'descripcion_features'] = 'VISTA'
descFeatures.loc[descFeatures['descripcion'].str.contains('VISTA', na=False), 'descripcion_features'] = 'VISTA'
descFeatures.loc[descFeatures['descripcion'].str.contains('UBICACION', na=False), 'descripcion_features'] = 'UBICACION'
descFeatures.loc[descFeatures['descripcion'].str.contains('UBICADO', na=False), 'descripcion_features'] = 'UBICACION'
descFeatures.loc[descFeatures['descripcion'].str.contains('TRANQUILA', na=False), 'descripcion_features'] = 'TRANQUILO'
descFeatures.loc[descFeatures['descripcion'].str.contains('TRANQUILO', na=False), 'descripcion_features'] = 'TRANQUILO'
descFeatures.loc[descFeatures['descripcion'].str.contains('TRANQUILIDAD', na=False), 'descripcion_features'] = 'TRANQUILO'
descFeatures.loc[descFeatures['descripcion'].str.contains('UNIVERSIDAD', na=False), 'descripcion_features'] = 'UNIVERSIDAD'
descFeatures.loc[descFeatures['descripcion'].str.contains('AVENIDA', na=False), 'descripcion_features'] = 'AVENIDA'
descFeatures.loc[descFeatures['descripcion'].str.contains('ESQUINA', na=False), 'descripcion_features'] = 'ESQUINA'
descFeatures.loc[descFeatures['descripcion'].str.contains('FACHADA', na=False), 'descripcion_features'] = 'FRENTE'
descFeatures.loc[descFeatures['descripcion'].str.contains('FRENTE', na=False), 'descripcion_features'] = 'FRENTE'
descFeatures.loc[descFeatures['descripcion'].str.contains('REMODELADO', na=False), 'descripcion_features'] = 'REMODELADO'
descFeatures.loc[descFeatures['descripcion'].str.contains('REMODELADA', na=False), 'descripcion_features'] = 'REMODELADO'
descFeatures.loc[descFeatures['descripcion'].str.contains('MODERNA', na=False), 'descripcion_features'] = 'MODERNO'
descFeatures.loc[descFeatures['descripcion'].str.contains('EXCLUSIVA', na=False), 'descripcion_features'] = 'LUJO'
descFeatures.loc[descFeatures['descripcion'].str.contains('LUJO', na=False), 'descripcion_features'] = 'LUJO'
descFeatures.loc[descFeatures['descripcion'].str.contains('CALIDAD', na=False), 'descripcion_features'] = 'LUJO'
descFeatures.loc[descFeatures['descripcion'].str.contains('MARMOL', na=False), 'descripcion_features'] = 'LUJO'
descFeatures.loc[descFeatures['descripcion'].str.contains('PORCELANATO', na=False), 'descripcion_features'] = 'LUJO'
descFeatures.loc[descFeatures['descripcion'].str.contains('JACUZZI', na=False), 'descripcion_features'] = 'LUJO'
descFeatures.loc[descFeatures['descripcion'].str.contains('VESTIDOR', na=False), 'descripcion_features'] = 'LUJO'
descFeatures.loc[descFeatures['descripcion'].str.contains('GOLF', na=False), 'descripcion_features'] = 'GOLF'
descFeatures.loc[descFeatures['descripcion'].str.contains('BODEGA', na=False), 'descripcion_features'] = 'BODEGA'
descFeatures.loc[descFeatures['descripcion'].str.contains('PLAYA', na=False), 'descripcion_features'] = 'PLAYA'
descFeatures.loc[descFeatures['descripcion'].str.contains('AMENIDADES ', na=False), 'descripcion_features'] = 'AMENIDADES'
descFeatures.loc[descFeatures['descripcion'].str.contains('FIESTAS', na=False), 'descripcion_features'] = 'AMENIDADES'
descFeatures.loc[descFeatures['descripcion'].str.contains('EVENTOS', na=False), 'descripcion_features'] = 'AMENIDADES'
descFeatures.loc[descFeatures['descripcion'].str.contains('LAVANDERIA', na=False), 'descripcion_features'] = 'AMENIDADES'
descFeatures.loc[descFeatures['descripcion'].str.contains('JUEGOS', na=False), 'descripcion_features'] = 'AMENIDADES'
descFeatures.loc[descFeatures['descripcion'].str.contains('SERVICIOS', na=False), 'descripcion_features'] = 'AMENIDADES'
descFeatures.loc[descFeatures['descripcion'].str.contains('SERVICIO', na=False), 'descripcion_features'] = 'AMENIDADES'
descFeatures.loc[descFeatures['descripcion'].str.contains('ASADOR', na=False), 'descripcion_features'] = 'AMENIDADES'
descFeatures.loc[descFeatures['descripcion'].str.contains('CANCHAS', na=False), 'descripcion_features'] = 'AMENIDADES'
descFeatures.loc[descFeatures['descripcion'].str.contains('TENIS', na=False), 'descripcion_features'] = 'AMENIDADES'
descFeatures.loc[descFeatures['descripcion'].str.contains('BARRA', na=False), 'descripcion_features'] = 'BARRA'
descFeatures.loc[descFeatures['descripcion'].str.contains('DESAYUNADORA', na=False), 'descripcion_features'] = 'BARRA'
descFeatures.loc[descFeatures['descripcion'].str.contains('DESAYUNADOR', na=False), 'descripcion_features'] = 'BARRA'
descFeatures.loc[descFeatures['descripcion'].str.contains('BALCON', na=False), 'descripcion_features'] = 'BALCON'
descFeatures.loc[descFeatures['descripcion'].str.contains('RECIBIDOR', na=False), 'descripcion_features'] = 'RECIBIDOR'
descFeatures.loc[descFeatures['descripcion'].str.contains('VESTIBULO', na=False), 'descripcion_features'] = 'RECIBIDOR'
descFeatures.loc[descFeatures['descripcion'].str.contains('CHIMENEA', na=False), 'descripcion_features'] = 'CHIMENEA'
descFeatures.loc[descFeatures['descripcion'].str.contains('DESPENSA', na=False), 'descripcion_features'] = 'DESPENSA'
descFeatures.loc[descFeatures['descripcion'].str.contains('LAVADERO', na=False), 'descripcion_features'] = 'LAVADERO'
descFeatures.loc[descFeatures['descripcion'].str.contains('REJA', na=False), 'descripcion_features'] = 'REJA'
descFeatures.loc[descFeatures['descripcion'].str.contains('BOILER', na=False), 'descripcion_features'] = 'CISTERNA'
descFeatures.loc[descFeatures['descripcion'].str.contains('CISTERNA', na=False), 'descripcion_features'] = 'CISTERNA'
descFeatures.loc[descFeatures['descripcion'].str.contains('AZOTEA', na=False), 'descripcion_features'] = 'TERRAZA'
descFeatures.loc[descFeatures['descripcion'].str.contains('TERRAZA', na=False), 'descripcion_features'] = 'TERRAZA'

#La tipamos
descFeatures['descripcion_features'] = descFeatures['descripcion_features'].astype('category')

#La guardamos con label encoding
cat_features_desc = ['descripcion_features']
encoder_desc = LabelEncoder()
encoded_desc = descFeatures[cat_features_desc].apply(encoder_desc.fit_transform)
encoded_desc['id'] = test['id']
encoded_desc = encoded_desc.rename(columns={'descripcion_features': 'descripcion_features_enc'})

#Agregamos la columna al los DF
descFeatures = pd.merge(descFeatures,encoded_desc, on='id', how='inner')
testProcesado = pd.merge(testProcesado,descFeatures, on='id', how='inner')
testProcesado = testProcesado.drop('descripcion',1)
testProcesado = testProcesado.drop('descripcion_features',1)
testProcesado = testProcesado.drop('descripcion_features_enc',1)

In [89]:
#OR entre las columnas repetidas en ambos DF
#Propiedad nueva
testProcesado['NUEVA'] = testProcesado['NUEVA_x'] | testProcesado['NUEVA_y']
#Calificativo (no aporta mucho)
testProcesado['EXCELENTE'] = testProcesado['EXCELENTE_x'] | testProcesado['EXCELENTE_y']
#Tipo de aviso
testProcesado['VENTA'] = testProcesado['VENTA_x'] | testProcesado['VENTA_y'] 
testProcesado['PREVENTA'] = testProcesado['PREVENTA_x'] | testProcesado['PREVENTA_y']
testProcesado['RENTA'] = testProcesado['RENTA_x'] | testProcesado['RENTA_y'] 
testProcesado['CREDITO'] = testProcesado['CREDITO_x'] | testProcesado['CREDITO_y']
testProcesado['REMATE'] = testProcesado['REMATE_x'] | testProcesado['REMATE_y']
testProcesado['OPORTUNIDAD'] = testProcesado['OPORTUNIDAD_x'] | testProcesado['OPORTUNIDAD_y']
#Tipo de propiedad -extra
testProcesado['RESIDENCIA'] = testProcesado['RESIDENCIA_x'] | testProcesado['RESIDENCIA_y']
testProcesado['EDIFICIO'] = testProcesado['EDIFICIO_x'] | testProcesado['EDIFICIO_y'] 
testProcesado['TORRE'] = testProcesado['TORRE_x'] | testProcesado['TORRE_y']
testProcesado['PISO'] = testProcesado['PISO_x'] | testProcesado['PISO_y']
testProcesado['LOFT'] = testProcesado['LOFT_x'] | testProcesado['LOFT_y']
testProcesado['FRACCIONAMIENTO'] = testProcesado['FRACCIONAMIENTO_x'] | testProcesado['FRACCIONAMIENTO_y']
testProcesado['DUPLEX'] = testProcesado['DUPLEX_x'] | testProcesado['DUPLEX_y'] 
testProcesado['COUNTRY'] = testProcesado['COUNTRY_x'] | testProcesado['COUNTRY_y']
testProcesado['QUINTA'] = testProcesado['QUINTA_x'] | testProcesado['QUINTA_y']
testProcesado['OFICINA'] = testProcesado['OFICINA_x'] | testProcesado['OFICINA_y']
testProcesado['PENTHOUSE'] = testProcesado['PENTHOUSE_x'] | testProcesado['PENTHOUSE_y']
#Tipo de propiedad -extra 2
testProcesado['REMODELADO'] = testProcesado['REMODELADO_x'] | testProcesado['REMODELADO_y']
testProcesado['MODERNO'] = testProcesado['MODERNO_x'] | testProcesado['MODERNO_y']
testProcesado['MINIMALISTA'] = testProcesado['MINIMALISTA_x'] | testProcesado['MINIMALISTA_y']
testProcesado['EQUIPADA'] = testProcesado['EQUIPADA_x'] | testProcesado['EQUIPADA_y']
#Jardin
testProcesado['JARDIN'] = testProcesado['JARDIN_x'] | testProcesado['JARDIN_y'] 
#Ubicacion
testProcesado['FRENTE'] = testProcesado['FRENTE_x'] | testProcesado['FRENTE_y'] 
testProcesado['ESQUINA'] = testProcesado['ESQUINA_x'] | testProcesado['ESQUINA_y'] 
testProcesado['CENTRO'] = testProcesado['CENTRO_x'] | testProcesado['CENTRO_y']
testProcesado['UBICACION'] = testProcesado['UBICACION_x'] | testProcesado['UBICACION_y']
#Vistas o panoramicas
testProcesado['VISTA'] = testProcesado['VISTA_x'] | testProcesado['VISTA_y']
#Caracteristicas extra
testProcesado['LUJO'] = testProcesado['LUJO_x'] |  testProcesado['LUJO_y']
testProcesado['BODEGA'] = testProcesado['BODEGA_x'] | testProcesado['BODEGA_y']
testProcesado['CAMPESTRE'] = testProcesado['CAMPESTRE_x'] | testProcesado['CAMPESTRE_x']
testProcesado['RANCHO'] = testProcesado['RANCHO_x'] | testProcesado['RANCHO_y'] 
testProcesado['PLAYA'] = testProcesado['PLAYA_x'] | testProcesado['PLAYA_y']
testProcesado['GOLF'] = testProcesado['GOLF_x'] | testProcesado['GOLF_y']
#Seguridad
testProcesado['SEGURIDAD'] = testProcesado['SEGURIDAD_x'] | testProcesado['SEGURIDAD_y'] 

#Borramos las columnas que quedaron del merge
testProcesado.drop(list(testProcesado.filter(regex = '_x')), axis = 1, inplace = True)
testProcesado.drop(list(testProcesado.filter(regex = '_y')), axis = 1, inplace = True)

In [90]:
testProcesado.drop('idzona', axis = 1, inplace = True)
testProcesado.drop('anio', axis = 1, inplace = True)

In [91]:
testProcesado.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 60000 entries, 0 to 59999
Data columns (total 72 columns):
id                            60000 non-null int64
habitaciones                  60000 non-null int64
garages                       60000 non-null int64
banos                         60000 non-null int64
gimnasio                      60000 non-null int64
usosmultiples                 60000 non-null int64
piscina                       60000 non-null int64
escuelascercanas              60000 non-null int64
centroscomercialescercanos    60000 non-null int64
2012                          60000 non-null int64
2013                          60000 non-null int64
2014                          60000 non-null int64
2015                          60000 non-null int64
2016                          60000 non-null int64
ciudad                        60000 non-null int64
provincia                     60000 non-null int64
tipodepropiedad               60000 non-null int64
antiguedad_agrupada     

In [92]:
testProcesado['mean'].fillna(testProcesado['mean'].mean(), inplace = True)
testProcesado['median'].fillna(testProcesado['median'].mean(), inplace = True)

In [93]:
#Exportamos el DF a CSV, para usar como entrada de los distintos algoritmos probados
testProcesado.to_csv('DATA/testProcesado_patov5.csv', index=False)

In [94]:
trainProcesado.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 240000 entries, 0 to 239999
Data columns (total 73 columns):
id                            240000 non-null int64
habitaciones                  240000 non-null int64
garages                       240000 non-null int64
banos                         240000 non-null int64
gimnasio                      240000 non-null int64
usosmultiples                 240000 non-null int64
piscina                       240000 non-null int64
escuelascercanas              240000 non-null int64
centroscomercialescercanos    240000 non-null int64
2012                          240000 non-null int64
2013                          240000 non-null int64
2014                          240000 non-null int64
2015                          240000 non-null int64
2016                          240000 non-null int64
precio                        240000 non-null float64
ciudad                        240000 non-null int64
provincia                     240000 non-null int64
tip

In [95]:
testProcesado.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 60000 entries, 0 to 59999
Data columns (total 72 columns):
id                            60000 non-null int64
habitaciones                  60000 non-null int64
garages                       60000 non-null int64
banos                         60000 non-null int64
gimnasio                      60000 non-null int64
usosmultiples                 60000 non-null int64
piscina                       60000 non-null int64
escuelascercanas              60000 non-null int64
centroscomercialescercanos    60000 non-null int64
2012                          60000 non-null int64
2013                          60000 non-null int64
2014                          60000 non-null int64
2015                          60000 non-null int64
2016                          60000 non-null int64
ciudad                        60000 non-null int64
provincia                     60000 non-null int64
tipodepropiedad               60000 non-null int64
antiguedad_agrupada     